# Imports

In [ ]:
# General includes.
import os
import io
import re
import errno
import gc
import random
import threading
import math
import itertools
import functools
from copy import deepcopy
import logging
import pickle
import hashlib

#from termcolor import colored, cprint
import colored
from datetime import datetime, timedelta
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from matplotlib import pyplot as plt

# Typing includes.
from typing import Dict, List, Optional, Any, Tuple, Callable, Iterable

# Numerical includes.
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# pyHealth includes.
from pyhealth.datasets import BaseDataset, MIMIC3Dataset, eICUDataset, SampleDataset, split_by_patient
from pyhealth.datasets.utils import MODULE_CACHE_PATH, strptime, hash_str
from pyhealth.data import Patient, Visit, Event
from pyhealth.data import Event, Visit, Patient

In [ ]:
# Model imports 
from transformers import BertTokenizer, BertModel, BertForMaskedLM, BertConfig, BertTokenizerFast
from transformers import TensorType
from transformers import AutoTokenizer, AutoConfig, AutoModel

In [ ]:
# Local imports
from trainlib import Trainer
from models import ModelA, EHRModel, CembRNN, DembRNN
from datasets import SimpleDataset, DatasetCacher

In [ ]:
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

# Globals

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
USE_GPU_ = False
BERT_USE_GPU_ = True  # BERT embeddings 
DEV_ = True  # Uses a small subset of MIMIC data: https://pyhealth.readthedocs.io/en/latest/api/datasets/pyhealth.datasets.MIMIC3Dataset.html#pyhealth.datasets.MIMIC3Dataset
GPU_STR_ = 'cuda'
# DATA_DIR_ = os.path.join(os.getcwd(), DATA_DIR_)
MIMIC_DATA_DIR_ = '~/sw/physionet.org/files/mimiciii/1.4'
EICU_DATA_DIR_ = '~/sw/eicu-collaborative-research-database-2.0/eicu-collaborative-research-database-2.0/'
BATCH_SIZE_ = 32
EMBEDDING_DIM_ = 264  # BERT requires a multiple of 12
SHUFFLE_ = True
SAMPLE_MULTIPLIER_ = 1

# TBD cache for BERT embeddings.
CACHE_DIR_ = 'cache'

# Set seed for reproducibility.
seed = 90210
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

def load_pickle(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)


def save_pickle(data, filename):
    with open(filename, "wb") as f:
        pickle.dump(data, f)
        
# https://stackoverflow.com/questions/50888391/pickle-of-object-with-getattr-method-in-python-returns-typeerror-object-no
class DotArgs(dict):
    """
    Access dictionary attributes via dot notation
    """
    def __getstate__(self):
        return vars(self)

    def __setstate__(self, state):
        vars(self).update(state)
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


# Preprocessing

### Load MIMIC III Data

In [ ]:
from pyhealth.medcode import InnerMap, ICD9CM

icd9cm = InnerMap.load("ICD9CM")
icd9cm.lookup("428.0") # get detailed info
icd9cm.get_ancestors("428.0") # get parents

print(icd9cm.lookup("78951")) # get detailed info
print(f'78951 ancestors {icd9cm.get_ancestors("78951")}') # get parents

print(icd9cm.lookup("7895")) # get detailed info
print(f'7895 ancestors {icd9cm.get_ancestors("7895")}') # get parents


print(icd9cm.lookup("7894")) # get detailed info
print(f'7894 ancestors {icd9cm.get_ancestors("7894")}') # get parents

print(icd9cm.lookup("78941")) # get detailed info
print(f'78941 ancestors {icd9cm.get_ancestors("78941")}') # get parents

print(ICD9CM.standardize('78951'))
print(ICD9CM.standardize('7895'))

In [ ]:
def _compute_duration_minutes(start_datetime: str, end_datetime: str) -> float:
    '''Return duration in minutes as a float.
    '''
    # MIMIC-III uses the following format: 2146-07-22 00:00:00
    start = datetime.strptime(start_datetime, '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime(end_datetime,   '%Y-%m-%d %H:%M:%S')
    return float((end - start).seconds)

class MIMIC3DatasetWrapper(MIMIC3Dataset):
    ''' Add extra tables to the MIMIC III dataset.
    
      Some of the tables we need like "D_ICD_DIAGNOSES", "D_ITEMS", "D_ICD_PROCEDURES"
      are not supported out of the box. 
      
      This class defines parsing methods to extract text data from these extra tables.
      The text data is generally joined on the PATIENTID, HADMID, ITEMID to match the
      pyHealth Vists class representation.
    '''
   
    # We need to add storage for text-based lookup tables here.
    def __init__(self, *args, **kwargs):
        self._valid_text_tables = ["D_ICD_DIAGNOSES", "D_ITEMS", "D_ICD_PROCEDURES", "D_LABITEMS"]
        self._text_descriptions = {x: {} for x in self._valid_text_tables}
        self._text_luts = {x: {} for x in self._valid_text_tables}
        self.refresh_cache = False
        # The pyHealth dataset cache doesn't know about this class's private members.
        if 'refresh_cache' in kwargs:
            self.refresh_cache = kwargs['refresh_cache']
        super().__init__(*args, **kwargs)
        self._do_cache()
    
    def _do_cache(self):
        '''The pyHealth dataset cache doesn't know about this classes private members.
        
          We need to wrap the caching function so it's aware of the additional luts
          to be saved/restored. The superclass is still responsible for saving/restoring
          `self.patients` from the DB.
        '''
        self.extra_filepath = ''.join([
            os.path.splitext(self.filepath)[0],
            '_dev' if self.dev else '',
            '_extras.pkl',
        ])
        
        # check if cache exists or refresh_cache is True
        if os.path.exists(self.extra_filepath) and (not self.refresh_cache):
            # load from cache
            logger.info(
                f"Loaded {self.dataset_name} base dataset from {self.extra_filepath}"
            )
            from_pickle = load_pickle(self.extra_filepath)
            self._valid_text_tables = from_pickle['_valid_text_tables']
            self._text_descriptions = from_pickle['_text_descriptions']
            self._text_luts = from_pickle['_text_luts']
        else:
            # load from raw data
            logger.info(f"Processing {self.dataset_name} base dataset...")
            to_cache = {
                '_valid_text_tables': self._valid_text_tables,
                '_text_descriptions': self._text_descriptions,
                '_text_luts': self._text_luts,
            }
            logger.info(f"Saved {self.dataset_name} base dataset to {self.extra_filepath}")
            save_pickle(to_cache, self.extra_filepath)
    
    def get_all_tables(self) -> List[str]: 
        return list(self._text_descriptions.keys())
        
    def get_text_dict(self, table_name: str) -> Dict[str, Dict[Any, Any]]:
        return self._text_descriptions.get(table_name)
    
    def set_text_lut(self, table_name: str, lut: Dict[Any, Any]) -> None:
        self._text_luts[table_name] = lut
    
    def get_text_lut(self, table_name: str) -> Dict[Any, Any]:
        return self._text_luts[table_name]
    
    def _add_events_to_patient_dict(
        self,
        patient_dict: Dict[str, Patient],
        group_df: pd.DataFrame,
    ) -> Dict[str, Patient]:
        #TODO(botelho3) Imported from PyHealth Base dataset githubf to
        #support parse_prescription
        """Helper function which adds the events column of a df.groupby object to the patient dict.
        
        Will be called at the end of each `self.parse_[table_name]()` function.
        Args:
            patient_dict: a dict mapping patient_id to `Patient` object.
            group_df: a df.groupby object, having two columns: patient_id and events.
                - the patient_id column is the index of the patient
                - the events column is a list of <Event> objects
        Returns:
            The updated patient dict.
        """
        for _, events in group_df.items():
            for event in events:
                patient_dict = self._add_event_to_patient_dict(patient_dict, event)
        return patient_dict

    
    def parse_prescriptions(self, patients: Dict[str, Patient]) -> Dict[str, Patient]:
        """Helper function which parses PRESCRIPTIONS table.
        
        TODO(botelho3) - we have to override this to include the text fields. The
        prescriptions table does not link to a separate D_ICD_* table in MIMIC-III
        thtat contains text descriptions of the prescription. The text descriptions
        are in the columns of this table. Regular pyHealth ignores these columns. We
        override this method to appent pyHealth Event objects containing the text
        columns to each patient.
        
        Will be called in `self.parse_tables()`
        Docs:
            - PRESCRIPTIONS: https://mimic.mit.edu/docs/iii/tables/prescriptions/
        Args:
            patients: a dict of `Patient` objects indexed by patient_id.
        Returns:
            The updated patients dict.
        """
        table = "PRESCRIPTIONS"
        # read table
        df = pd.read_csv(
            os.path.join(self.root, f"{table}.csv"),
            low_memory=False,
            dtype={"SUBJECT_ID": str, "HADM_ID": str, "NDC": str,
                   "DRUG_TYPE": str, "DRUG": str,
                   "PROD_STRENGTH": str, "ROUTE": str, "ENDDATE": str},
        )
        # drop records of the other patients
        df = df[df["SUBJECT_ID"].isin(patients.keys())]
        df = df.dropna()
        # sort by start date and end date
        df = df.sort_values(
            ["SUBJECT_ID", "HADM_ID", "STARTDATE", "ENDDATE"], ascending=True
        )
        # group by patient and visit
        group_df = df.groupby("SUBJECT_ID")
        
        # parallel unit for prescription (per patient)
        def prescription_unit(p_id, p_info):
            events = []
            for v_id, v_info in p_info.groupby("HADM_ID"):
                zipped = zip(v_info["STARTDATE"], v_info["NDC"], v_info["DRUG_TYPE"],
                             v_info["DRUG"], v_info["PROD_STRENGTH"], v_info["ROUTE"],
                             v_info["ENDDATE"])
                for startdate, code, dtype, dname, dose, route, enddate in zipped:
                    if not type(startdate) == str:
                        startdate = '2142-07-18 00:00:00'
                    if not type(enddate) == str:
                        enddate = '2142-07-18 00:00:00'
                    assert(type(dname) is str)
                    # if not type(enddate) is str:
                    #     print(f'Not matching enddate {enddate} startdate {startdate}')
                    #     print(f'dname {dname}, hadm_id {v_id}, p_id {p_id}')
                    assert(type(startdate) is str)
                    assert(type(enddate) is str)
                    event = Event(
                        code=code,
                        table=table,
                        vocabulary="NDC",
                        visit_id=v_id,
                        patient_id=p_id,
                        timestamp=strptime(startdate),
                        dtype=dtype,
                        dname=dname,
                        dose=dose,
                        route=route,
                        duration=_compute_duration_minutes(startdate, enddate),
                    )
                    events.append(event)
            return events

                # parallel apply
        group_df = group_df.parallel_apply(
            lambda x: prescription_unit(x.SUBJECT_ID.unique()[0], x)
        )

        patients = self._add_events_to_patient_dict(patients, group_df)
        return patients
    
    # Note the name has to match the table name exactly.
    # See https://github.com/sunlabuiuc/PyHealth/blob/master/pyhealth/datasets/mimic3.py#L71.
    def parse_d_icd_diagnoses(self, patients: Dict[str, Patient]) -> Dict[str, Patient]: 
        """Helper function which parses D_ICD_DIAGNOSIS table.
        Will be called in `self.parse_tables()`
        Docs:
            - D_ICD_DIAGNOSIS: https://mimic.mit.edu/docs/iii/tables/d_icd_diagnoses/
        Args:
            patients: a dict of `Patient` objects indexed by patient_id.
        Returns:
            The unchanged patients dict.
        Note:
            This function doesn't up date the patients dict like other part_*() functions.
            Here we read the D_ICD_DIAGNOSES.csv file containing ICD9_code -> text mappings
            and store them in a dict `self._text_descriptions[table]`.
            
            The dict is used as a ICD9_code -> text diagnosis description lookup
            for DescEmb.
        """
        table = "D_ICD_DIAGNOSES"
        print(f"Parsing {table}")
        assert(table in self._valid_text_tables)
        
        
        # read table
        df = pd.read_csv(
            os.path.join(self.root, f"{table}.csv"),
            usecols=["ICD9_CODE", "SHORT_TITLE", "LONG_TITLE"],
            dtype={"ICD9_CODE": str, "SHORT_TITLE": str, "LONG_TITLE": str}
        )
        
        # drop rows with missing values
        df = df.dropna(subset=["ICD9_CODE", "SHORT_TITLE", "LONG_TITLE"])
        # sort by sequence number (i.e., priority)
        df = df.sort_values(["ICD9_CODE"], ascending=True)
       
        # print(df.head())
        self._text_descriptions[table] = df.reset_index(drop=True).to_dict(orient='split')
        
        # We haven't altered the patients array, just return it.
        return patients
    
    def parse_d_labitems(self, patients: Dict[str, Patient]) -> Dict[str, Patient]: 
        """Helper function which parses D_LABITEMS table.
        Will be called in `self.parse_tables()`
        Docs:
            - D_LABITEMS: https://mimic.mit.edu/docs/iii/tables/d_labitems/
        Args:
            patients: a dict of `Patient` objects indexed by patient_id.
        Returns:
            The unchanged patients dict.
        Note:
            This function doesn't up date the patients dict like other part_*() functions.
            Here we read the D_LABITEMS.csv file containing ICD9_code -> text mappings
            and store them in a dict `self._text_descriptions[table]`.
            
            The dict is used as a ICD9_code -> text lab measurement description lookup
            for DescEmb.
        """
        table = "D_LABITEMS"
        print(f"Parsing {table}")
        assert(table in self._valid_text_tables)
        
        # read table
        df = pd.read_csv(
            os.path.join(self.root, f"{table}.csv"),
            usecols=["ITEMID", "LABEL", "CATEGORY", "FLUID"],
            dtype={"ITEMID": str, "LABEL": str, "CATEGORY": str, "FLUID": str}
        )
        
        # drop rows with missing values
        df = df.dropna(subset=["ITEMID", "LABEL", "CATEGORY", "FLUID"])
        # sort by sequence number (i.e., priority)
        df = df.sort_values(["ITEMID"], ascending=True)
       
        self._text_descriptions[table] = df.reset_index(drop=True).to_dict(orient='split')
        
        # We haven't altered the patients array, just return it.
        return patients
    
    
    def parse_d_items(self, patients: Dict[str, Patient]) -> Dict[str, Patient]: 
        # TODO(botelho3) - Note this may not be totally useable because the ITEMID
        # uinqiue key only links to these tables using ITEMID
        #   - INPUTEVENTS_MV 
        #   - OUTPUTEVENTS on ITEMID
        #   - PROCEDUREEVENTS_MV on ITEMID
        # 
        # Not to the tables we want e.g. 
        """Helper function which parses D_ITEMS table.
        Will be called in `self.parse_tables()`
        Docs:
            - D_ITEMS: https://mimic.mit.edu/docs/iii/tables/d_items/
        Args:
            patients: a dict of `Patient` objects indexed by patient_id.
        Returns:
            The unchanged patients dict.
        Note:
            This function doesn't up date the patients dict like other part_*() functions.
            Here we read the D_ITEMS.csv file containing ICD9_code -> text mappings
            and store them in a dict `self._text_descriptions[table]`.
            
            The dict is used as a ICD9_code -> text inputs/output/procedure events lookup
            for DescEmb.
        """
        table = "D_ITEMS"
        print(f"Parsing {table}")
        assert(table in self._valid_text_tables)
        
        # read table
        df = pd.read_csv(
            os.path.join(self.root, f"{table}.csv"),
            usecols=["ITEMID", "LABEL", "CATEGORY"],
            dtype={"ITEMID": str, "LABEL": str, "CATEGORY": str}
        )
        
        # drop rows with missing values
        df = df.dropna(subset=["ITEMID", "LABEL", "CATEGORY"])
        # sort by sequence number (i.e., priority)
        df = df.sort_values(["ITEMID"], ascending=True)
       
        self._text_descriptions[table] = df.reset_index(drop=True).to_dict(orient='split')
        
        # We haven't altered the patients array, just return it.
        return patients
    
    
    def parse_d_icd_procedures(self, patients: Dict[str, Patient]) -> Dict[str, Patient]: 
        """Helper function which parses D_ICD_PROCEDURES table.
        
        Will be called in `self.parse_tables()`
        Docs:
            - D_ICD_PROCEDURES: https://mimic.mit.edu/docs/iii/tables/d_icd_procedures/
        Args:
            patients: a dict of `Patient` objects indexed by patient_id.
        Returns:
            The unchanged patients dict.
        Note:
            This function doesn't up date the patients dict like other part_*() functions.
            Here we read the D_ICD_PROCEDURES.csv file containing ICD9_code -> text mappings
            and store them in a dict `self._text_descriptions[table]`.
            
            The dict is used as a ICD9_code -> text procedure description lookup for DescEmb.
        """
        table = "D_ICD_PROCEDURES"
        print(f"Parsing {table}")
        assert(table in self._valid_text_tables)
        
        # read table
        df = pd.read_csv(
            os.path.join(self.root, f"{table}.csv"),
            usecols=["ICD9_CODE", "SHORT_TITLE", "LONG_TITLE"],
            dtype={"ICD9_CODE": str, "SHORT_TITLE": str, "LONG_TITLE": str}
        )
        
        # drop rows with missing values
        df = df.dropna(subset=["ICD9_CODE", "SHORT_TITLE", "LONG_TITLE"])
        # sort by sequence number (i.e., priority)
        df = df.sort_values(["ICD9_CODE"], ascending=True)
       
        # print(df.head())
        self._text_descriptions[table] = df.reset_index(drop=True).to_dict(orient='split')
        
        # We haven't altered the patients array, just return it.
        return patients
    

In [ ]:
print(f'Reading data from: `{MIMIC_DATA_DIR_}`')

mimic3base = MIMIC3DatasetWrapper(
    # root="https://storage.googleapis.com/pyhealth/Synthetic_MIMIC-III/",
    root=MIMIC_DATA_DIR_,
    dataset_name='mimic_3_dataset',
    tables=["D_ICD_DIAGNOSES", "D_ICD_PROCEDURES", "D_ITEMS", "D_LABITEMS",
            "DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"], # "LABEVENTS"],
    # map all NDC codes to ATC 3-rd level codes in these tables
    # See https://en.wikipedia.org/wiki/Anatomical_Therapeutic_Chemical_Classification_System.
    code_mapping={"NDC": ("ATC", {"target_kwargs": {"level": 3}})},
    # Reads a subset of the data. Disable for full training run.
    dev = DEV_,
    # True = Slow, rebuilds the dataset instead of caching.
    refresh_cache=False,
)

mimic3base.stat()
mimic3base.info()


In [ ]:
table_names = mimic3base.get_all_tables()
print(table_names)

print('\033[92m' '====Tables====\n' '\033[0m')
# print(colored('====Tables====\n', 'green'))
# print(colored.fg('green') + '====Tables====\n')
for t in table_names:
    d = mimic3base.get_text_dict(t)
    print(f"Table: {t}")
    print(d['data'][:5])
    print('\n\n')

# Take the cached tables from the parse_tables function and build the {ICD9 -> (short_name, long_name)}
# lookup tables.
for t in table_names:
    d = mimic3base.get_text_dict(t)
    d = d['data']
    lut = {record[0]: record[1:] for record in d}
    mimic3base.set_text_lut(t,  lut)
    
print('\033[92m' '====Luts====\n' '\033[0m')
# print(f'{colored.fg("green")} ====Luts====\n')
for t in table_names:
    d = mimic3base.get_text_lut(t)
    print(f"Lut {t}:\n{dict(itertools.islice(d.items(), 2))}")

### Tasks

Declare tasks for 2 of the 5 prediction tasks specified in the paper. We will create dataloaders for each task that contain the ICD codes and the raw text for each (patient, visit).

#### CodeEMB Pred tasks

In [ ]:
# The original authors tackled 5 tasks
#   1. readmission
#   2. mortality
#   3. an ICU stay exceeding three days
#   4. an ICU stay exceeding seven days
#   5. diagnosis prediction

# def readmission_pred_task(patient, time_window=3):
#     """
#     patient is a <pyhealth.data.Patient> object
#     """
#     samples = []

#     # we will drop the last visit
#     for i in range(len(patient) - 1):
#         visit: Visit = patient[i]
#         next_visit: Visit = patient[i + 1]

#         # step 1: define the readmission label 
#         # get time difference between current visit and next visit
#         time_diff = (next_visit.encounter_time - visit.encounter_time).days
#         readmission_label = 1 if time_diff < time_window else 0
        
#         # step 2: get code-based feature information
#         conditions = visit.get_code_list(table="DIAGNOSES_ICD")
#         procedures = visit.get_code_list(table="PROCEDURES_ICD")
#         drugs = visit.get_code_list(table="PRESCRIPTIONS")
#         drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        
#         # step 3: exclusion criteria: visits without condition, procedure, or drug
#         if len(conditions) * len(procedures) == 0 * len(drugs_full) == 0:
#             continue
#         if len(conditions) + len(procedures) + len(drugs_full) < 5:
#             # Exclude stays with less than 5 procedures.
#             continue
        
#         # step 3.5: build text lists from the ICD codes
#         diag_lut = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
#         proc_lut = mimic3base.get_text_lut("D_ICD_PROCEDURES")
#         conditions_text = [diag_lut.get(cond,("", ""))[1] for cond in conditions]
#         procedures_text = [proc_lut.get(proc,("", ""))[1] for proc in procedures]
#         drugs_text = [' '.join([d['dname'], d['dtype'], d['dose'], d['route'], str(d['duration'])])
#                       for d in [d.attr_dict for d in drugs_full]]
            
#         # step 4: assemble the samples
#         # TODO: should also exclude visit with age < 18
#         samples.append(
#             {
#                 "visit_id": visit.visit_id,
#                 "patient_id": patient.patient_id,
#                 # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
#                 "conditions": conditions,
#                 "procedures": procedures,
#                 "conditions_text": conditions_text,
#                 "procedures_text": procedures_text,
#                 "drugs": drugs,
#                 "drugs_text": drugs_text,
#                 # "labevents": labevents,
#                 # "labevents_text": labevents_text
#                 "label": readmission_label,
#             }
#         )
#     # no cohort selection
#     return samples


# def mortality_pred_task(MORTALITY_PER_VISIT_ICD_9_CODE_COUNT_, patient):
#     """
#     patient is a <pyhealth.data.Patient> object
#     """
#     samples = []

#     # loop over all visits but the last one
#     for i in range(len(patient) - 1):

#         # visit and next_visit are both <pyhealth.data.Visit> objects
#         # there are no vists.attr_dict keys
#         visit = patient[i]
#         next_visit = patient[i + 1]
#         # if i == 0: print(visit)

#         # step 1: define the mortality_label
#         if next_visit.discharge_status not in [0, 1]:
#             mortality_label = 0
#         else:
#             mortality_label = int(next_visit.discharge_status)

#         # step 2: get code-based feature information
#         conditions = visit.get_code_list(table="DIAGNOSES_ICD")
#         procedures = visit.get_code_list(table="PROCEDURES_ICD")
#         drugs = visit.get_code_list(table="PRESCRIPTIONS")
#         drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
#         # if i == 0: print([d.attr_dict for d in drugs_full])
#         # if i == 0: print(conditions)
#         # if i == 0: print(procedures)
#         # if i == 0: print(drugs)
#         # TODO(botelho3) - add this datasource back in once we have full MIMIC-III dataset.
#         # labevents = visit.get_code_list(table="LABEVENTS")

#         # step 3: exclusion criteria: visits without condition, procedure, or drug
#         if len(conditions) * len(procedures) == 0 * len(drugs_full) == 0:
#             continue
#         if len(conditions) + len(procedures) + len(drugs_full) < 5:
#             # Exclude stays with less than 5 procedures.
#             continue
        
#         # step 3.5: build text lists from the ICD codes
#         diag_lut = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
#         proc_lut = mimic3base.get_text_lut("D_ICD_PROCEDURES")
#         # if i == 0: print(diag_lut)
#         # if i == 0: print(proc_lut)
#         # Index 0 is shortname, index 1 is longname.
#         # print([str(cond) + ' ' + str(diag_lut.get(cond)) for cond in conditions])
#         # print(proc_lut.get(procedures[0]))
#         conditions_text = [diag_lut.get(cond,("", ""))[1] for cond in conditions]
#         procedures_text = [proc_lut.get(proc,("", ""))[1] for proc in procedures]
#         drugs_text = [' '.join([d['dname'], d['dtype'], d['dose'], d['route'], str(d['duration'])])
#                       for d in [d.attr_dict for d in drugs_full]]
#         # TODO(botelho3) - add the labevents data source back in once we have full MIMIC-III dataset.
#         # labevents_text =
        
#         # step 4: assemble the samples
#         samples.append(
#             {
#                 "visit_id": visit.visit_id,
#                 "patient_id": patient.patient_id,
#                 # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
#                 "conditions": conditions,
#                 "procedures": procedures,
#                 "conditions_text": conditions_text,
#                 "procedures_text": procedures_text,
#                 "drugs": drugs,
#                 "drugs_text": drugs_text,
#                 # "labevents": labevents,
#                 # "labevents_text": labevents_text
#                 "label": mortality_label,
#             }
#         )
#     return samples

def readmission_pred_task_cemb(CODE_COUNT, patient):
    """
    patient is a <pyhealth.data.Patient> object
    each sample is a list of vists for 1 patient.
    """
   
    # TODO(botelho3) - stupid hack around the limitations of SampleDataset validator.
    kMaxVisits = 5
    if len(patient) < 1 or len(patient) > kMaxVisits:
        return []
        
    samples = [{
        "visit_id": patient[0].visit_id,
        "patient_id": patient.patient_id,
        # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
        "num_visits": 0,
        "conditions": [[] for v in range(kMaxVisits)],
        "procedures": [[] for v in range(kMaxVisits)],
        "conditions_text": [[] for v in range(kMaxVisits)],
        "procedures_text": [[] for v in range(kMaxVisits)],
        "drugs": [[] for v in range(kMaxVisits)],
        "drugs_text": [[] for v in range(kMaxVisits)],
        # "labevents": labevents,
        # "labevents_text": labevents_text
        "label": 0,
    }]

    # They were admitted > 1 time.
    global_readmission_label = 1 if len(patient) > 1 else 0

    # loop over all visits
    out_idx = 0
    for i, visit in enumerate(patient):
        # visit and next_visit are both <pyhealth.data.Visit> objects
        # there are no vists.attr_dict keys

        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        drugs = visit.get_code_list(table="PRESCRIPTIONS")
        drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        # if i == 0: print([d.attr_dict for d in drugs_full])
        # if i == 0: print(conditions)
        # if i == 0: print(procedures)
        # if i == 0: print(drugs)
        # TODO(botelho3) - add this datasource back in once we have full MIMIC-III dataset.
        # labevents = visit.get_code_list(table="LABEVENTS")

        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) * len(drugs_full) == 0:
            continue
        if len(conditions) + len(procedures) + len(drugs_full) < 5:
            # Exclude stays with less than 5 procedures.
            continue
        
        # step 3.5: build text lists from the ICD codes
        diag_lut = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        proc_lut = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        # if i == 0: print(diag_lut)
        # if i == 0: print(proc_lut)
        # Index 0 is shortname, index 1 is longname.
        # print([str(cond) + ' ' + str(diag_lut.get(cond)) for cond in conditions])
        # print(proc_lut.get(procedures[0]))
        conditions_text = [diag_lut.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [proc_lut.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], str(d['dose']), d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
        # TODO(botelho3) - add the labevents data source back in once we have full MIMIC-III dataset.
        # labevents_text =
        
        # step 4: assemble the samples
        # Each field is a list of visits.
        samples[0]['num_visits'] = samples[0]['num_visits'] + 1
        samples[0]['conditions'][out_idx]=conditions
        samples[0]['procedures'][out_idx]=procedures
        samples[0]['conditions_text'][out_idx]= conditions_text
        samples[0]['procedures_text'][out_idx] = procedures_text
        samples[0]['drugs'][out_idx] = drugs
        samples[0]['drugs_text'][out_idx] = drugs_text
        out_idx = out_idx + 1
    samples[0]['label'] = global_readmission_label
   
    # Record all unique codes and their frequency for LUT.
    for visit in samples[0]['conditions']:
        for code in visit:
            CODE_COUNT[code] = CODE_COUNT.get(code, 0) + 1
           
    # If none of the samples met the criteria return an empty list.
    if samples[0]['num_visits'] == 0:
        return []

    # Potentially multiply the sample n-times to increase dataset size.
    samples.extend(
        list(deepcopy(samples[0]) for s in range(SAMPLE_MULTIPLIER_-1))
    )
        
    return samples
    
    
def mortality_pred_task_cemb(CODE_COUNT, patient):
    """
    patient is a <pyhealth.data.Patient> object
    each sample is a list of vists for 1 patient.
    """
   
    # TODO(botelho3) - stupid hack around the limitations of SampleDataset validator.
    kMaxVisits = 5
    if len(patient) < 1 or len(patient) > kMaxVisits:
        return []
        
    samples = [{
        "visit_id": patient[0].visit_id,
        "patient_id": patient.patient_id,
        # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
        "num_visits": 0,
        "conditions": [[] for v in range(kMaxVisits)],
        "procedures": [[] for v in range(kMaxVisits)],
        "conditions_text": [[] for v in range(kMaxVisits)],
        "procedures_text": [[] for v in range(kMaxVisits)],
        "drugs": [[] for v in range(kMaxVisits)],
        "drugs_text": [[] for v in range(kMaxVisits)],
        # "labevents": labevents,
        # "labevents_text": labevents_text
        "label": 0,
    }]
    
    global_mortality_label = 0
    # loop over all visits but the last one
    for i in range(len(patient)):

        # visit and next_visit are both <pyhealth.data.Visit> objects
        # there are no vists.attr_dict keys
        visit: Visit = patient[i]
        # next_visit: Visit = patient[i + 1]
        # if i == 0: print(visit)

        # step 1: define the mortality_label
        # if next_visit.discharge_status not in [0, 1]:
        #     mortality_label = 0
        # else:
        #     mortality_label = int(next_visit.discharge_status)
        mortality_label = int(visit.discharge_status)
        global_mortality_label |= mortality_label

    # loop over all visits
    out_idx = 0
    for i, visit in enumerate(patient):
        # visit and next_visit are both <pyhealth.data.Visit> objects
        # there are no vists.attr_dict keys

        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        drugs = visit.get_code_list(table="PRESCRIPTIONS")
        drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        # if i == 0: print([d.attr_dict for d in drugs_full])
        # if i == 0: print(conditions)
        # if i == 0: print(procedures)
        # if i == 0: print(drugs)
        # TODO(botelho3) - add this datasource back in once we have full MIMIC-III dataset.
        # labevents = visit.get_code_list(table="LABEVENTS")

        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) * len(drugs_full) == 0:
            continue
        if len(conditions) + len(procedures) + len(drugs_full) < 5:
            # Exclude stays with less than 5 procedures.
            continue
        
        # step 3.5: build text lists from the ICD codes
        diag_lut = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        proc_lut = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        # if i == 0: print(diag_lut)
        # if i == 0: print(proc_lut)
        # Index 0 is shortname, index 1 is longname.
        # print([str(cond) + ' ' + str(diag_lut.get(cond)) for cond in conditions])
        # print(proc_lut.get(procedures[0]))
        conditions_text = [diag_lut.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [proc_lut.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], str(d['dose']), d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
        # TODO(botelho3) - add the labevents data source back in once we have full MIMIC-III dataset.
        # labevents_text =
        
        # step 4: assemble the samples
        # Each field is a list of visits.
        samples[0]['num_visits'] = samples[0]['num_visits'] + 1
        samples[0]['conditions'][out_idx]=conditions
        samples[0]['procedures'][out_idx]=procedures
        samples[0]['conditions_text'][out_idx]= conditions_text
        samples[0]['procedures_text'][out_idx] = procedures_text
        samples[0]['drugs'][out_idx] = drugs
        samples[0]['drugs_text'][out_idx] = drugs_text
        out_idx = out_idx + 1
    samples[0]['label'] = global_mortality_label
   
    # Record all unique codes and their frequency for LUT.
    for visit in samples[0]['conditions']:
        for code in visit:
            CODE_COUNT[code] = CODE_COUNT.get(code, 0) + 1
           
    # If none of the samples met the criteria return an empty list.
    if samples[0]['num_visits'] == 0:
        return []

    # Potentially multiply the sample n-times to increase dataset size.
    samples.extend(
        list(deepcopy(samples[0]) for s in range(SAMPLE_MULTIPLIER_-1))
    )
        
    return samples


#### DescEmb Pred Tasks

In [ ]:
# The original authors tackled 5 tasks
#   1. readmission
#   2. mortality
#   3. an ICU stay exceeding three days
#   4. an ICU stay exceeding seven days
#   5. diagnosis prediction

#{code: idx for idx, code in enumerate(READMISSION_PER_PATIENT_ICD_9_CODE_COUNT_.keys())}
def readmission_pred_task_demb(CODE_COUNT, patient, time_window=3):
    """
    patient is a <pyhealth.data.Patient> object
    """
    samples = []
    visits = []
    kMaxListSize = 40
   
    # Length 1 patients by defn are not readmitted.
    if len(patient) < 1:
        return samples

    # we will drop the last visit
    global_readmission_label = 0
    global_readmission_label = 1 if len(patient) > 1 else 0
#     for i in range(len(patient) - 1):
#         visit: Visit = patient[i]
#         next_visit: Visit = patient[i + 1]

#         # step 1: define the readmission label 
#         # get time difference between current visit and next visit
#         time_diff = (next_visit.encounter_time - visit.encounter_time).days
#         readmission_label = 1 if time_diff < time_window else 0
#         global_readmission_label |= readmission_label  
       
    for i, visit in enumerate(patient):
        # visit: Visit = patient[i]
        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        # drugs = visit.get_code_list(table="PRESCRIPTIONS")
        drugs = [x.code for x in visit.get_event_list(table="PRESCRIPTIONS")]
        drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        
        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) == 0 * len(drugs_full) == 0:
            continue
        if len(conditions) + len(procedures) + len(drugs_full) < 5:
            # Exclude stays with less than 5 procedures.
            continue
        
        # step 3.5: build text lists from the ICD codes
        d_diag = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        d_proc = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        conditions_text = [d_diag.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [d_proc.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], d['dose'], d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
            
        # step 4: assemble the samples
        visits.append(
            {
                "visit_id": visit.visit_id,
                "patient_id": patient.patient_id,
                # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
                "conditions": conditions,
                "procedures": procedures,
                "conditions_text": conditions_text,
                "procedures_text": procedures_text,
                "drugs": drugs,
                "drugs_text": drugs_text,
                # "labevents": labevents,
                # "labevents_text": labevents_text
                "label": global_readmission_label,
            }
        )
        

    # Return empty list, didn't meet exclusion criteria.
    num_visits = len(visits)
    if num_visits < 1:
        return samples
    
    def pad_field(field, visits, empty_val: Any):
        l = [empty_val for x in range(kMaxListSize)]
        data = [x[field] for x in visits]
        data = list(itertools.chain.from_iterable(data))
        slice_size = min(kMaxListSize, len(data)) - 1
        l[:slice_size] = data[:slice_size]
        return l, slice_size 
    
    conditions, conditions_pad = pad_field("conditions", visits, '0')
    conditions_text, conditions_text_pad = pad_field("conditions_text", visits, '')
    procedures, procedures_pad = pad_field("procedures", visits, '0')
    procedures_text, procedures_text_pad = pad_field("procedures_text", visits, '')
    drugs, drugs_pad = pad_field("drugs", visits, '0')
    drugs_text, drugs_text_pad = pad_field("drugs_text", visits, '')
    assert(drugs_pad == drugs_text_pad)
    sample = {
        "patient_id": patient.patient_id,
        # TODO(botelho3) Why does pyhealth require a visit id in the keys if we're combining vists?
        "visit_id": visits[0]["visit_id"],
        "num_visits": num_visits,
        # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
        "conditions": conditions,
        "conditions_text": conditions_text,
        "procedures": procedures,
        "procedures_text": procedures_text,
        "drugs": drugs,
        "drugs_text": drugs_text,
        
        "conditions_pad": conditions_pad,
        "procedures_pad": procedures_pad,
        "conditions_text_pad": conditions_text_pad,
        "procedures_text_pad": procedures_text_pad,
        "drugs_pad": drugs_pad,
        "drugs_text_pad": drugs_text_pad,
        # "labevents": labevents,
        # "labevents_text": labevents_text
        "label": global_readmission_label,
    }
    # sample = {
    #     "patient_id": patient.patient_id,
    #     # TODO(botelho3) Why does pyhealth require a visit id in the keys if we're combining vists?
    #     "visit_id": visits[0]["visit_id"],
    #     "num_visits": num_visits,
    #     # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
    #     "conditions": [[x["conditions"] for x in visits]],
    #     "procedures": [[x["procedures"] for x in visits]],
    #     "conditions_text": [[x["conditions_text"] for x in visits]],
    #     "procedures_text": [[x["procedures_text"] for x in visits]],
    #     "drugs": [[[x["drugs"] for x in visits]]],
    #     "drugs_text": [[x["drugs_text"] for x in visits]],
    #     # "labevents": labevents,
    #     # "labevents_text": labevents_text
    #     "label": global_readmission_label,
    # }
    for code in sample['conditions']:
        CODE_COUNT[code] = CODE_COUNT.get(code, 0) + 1

    samples.extend(
        list(deepcopy(sample) for s in range(SAMPLE_MULTIPLIER_))
    )
    return samples


def mortality_pred_task_demb(CODE_COUNT, patient):
    """
    patient is a <pyhealth.data.Patient> object
    """
    samples = []
    visits = []
    kMaxListSize = 40

    global_mortality_label = 0
    # loop over all visits but the last one
    for i in range(len(patient)):

        # visit and next_visit are both <pyhealth.data.Visit> objects
        # there are no vists.attr_dict keys
        visit: Visit = patient[i]
        # next_visit: Visit = patient[i + 1]
        # if i == 0: print(visit)

        # step 1: define the mortality_label
        # if next_visit.discharge_status not in [0, 1]:
        #     mortality_label = 0
        # else:
        #     mortality_label = int(next_visit.discharge_status)
        mortality_label = int(visit.discharge_status)
        global_mortality_label |= mortality_label
        
    # loop over all visits but the last one
    for i, visit in enumerate(patient):
        # visit: Visit.
        
        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        # drugs = visit.get_code_list(table="PRESCRIPTIONS")
        drugs = [x.code for x in visit.get_event_list(table="PRESCRIPTIONS")]
        drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        # if i == 0: print([d.attr_dict for d in drugs_full])
        # if i == 0: print(conditions)
        # if i == 0: print(procedures)
        # if i == 0: print(drugs)
        # TODO(botelho3) - add this datasource back in once we have full MIMIC-III dataset.
        # labevents = visit.get_code_list(table="LABEVENTS")

        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) == 0 * len(drugs_full) == 0:
            continue
        if len(conditions) + len(procedures) + len(drugs_full) < 5:
            # Exclude stays with less than 5 procedures.
            continue
        
        # step 3.5: build text lists from the ICD codes
        diag_lut = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        proc_lut = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        # if i == 0: print(d_diag)
        # if i == 0: print(d_proc)
        # Index 0 is shortname, index 1 is longname.
        # print([str(cond) + ' ' + str(d_diag.get(cond)) for cond in conditions])
        # print(d_proc.get(procedures[0]))
        conditions_text = [diag_lut.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [proc_lut.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], d['dose'], d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
        # TODO(botelho3) - add the labevents data source back in once we have full MIMIC-III dataset.
        # labevents_text =
        
        # step 4: assemble the samples into a pyHealth Visit.
        visits.append(
            {
                "visit_id": visit.visit_id,
                "patient_id": patient.patient_id,
                # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
                "conditions": conditions,
                "procedures": procedures,
                "conditions_text": conditions_text,
                "procedures_text": procedures_text,
                "drugs": drugs,
                "drugs_text": drugs_text,
                # "labevents": labevents,
                # "labevents_text": labevents_text
                "label": global_mortality_label,
            }
        )
    
    # Return empty list, didn't meet exclusion criteria.
    num_visits = len(visits)
    if num_visits < 1:
        return [] 
   
    # pyHealth requires that all list fields in sample are equal size.
    def pad_field(field, visits, empty_val: Any):
        l = [empty_val for x in range(kMaxListSize)]
        data = [x[field] for x in visits]
        data = list(itertools.chain.from_iterable(data))
        slice_size = min(kMaxListSize, len(data))
        l[:slice_size] = data[:slice_size]
        return l, slice_size
    
    conditions, conditions_pad = pad_field("conditions", visits, '0')
    conditions_text, conditions_text_pad = pad_field("conditions_text", visits, '')
    procedures, procedures_pad = pad_field("procedures", visits, '0')
    procedures_text, procedures_text_pad = pad_field("procedures_text", visits, '')
    drugs, drugs_pad = pad_field("drugs", visits, '0')
    drugs_text, drugs_text_pad = pad_field("drugs_text", visits, '')
    assert(drugs_pad == drugs_text_pad)
    sample = {
        "patient_id": patient.patient_id,
        # TODO(botelho3) Why does pyhealth require a visit id in the keys if we're combining vists?
        "visit_id": visits[0]["visit_id"],
        "num_visits": num_visits,
        # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
        "conditions": conditions,
        "conditions_text": conditions_text,
        "procedures": procedures,
        "procedures_text": procedures_text,
        "drugs": drugs,
        "drugs_text": drugs_text,
        
        "conditions_pad": conditions_pad,
        "procedures_pad": procedures_pad,
        "conditions_text_pad": conditions_text_pad,
        "procedures_text_pad": procedures_text_pad,
        "drugs_pad": drugs_pad,
        "drugs_text_pad": drugs_text_pad,
        # "labevents": labevents,
        # "labevents_text": labevents_text
        "label": global_mortality_label,
    }
    # sample = {
    #     "patient_id": patient.patient_id,
    #     # TODO(botelho3) Why does pyhealth require a visit id in the keys if we're combining vists?
    #     "visit_id": visits[0]["visit_id"],
    #     "num_visits": num_visits,
    #     # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
    #     "conditions": [[x["conditions"] for x in visits]],
    #     "procedures": [[x["procedures"] for x in visits]],
    #     "conditions_text": [[x["conditions_text"] for x in visits]],
    #     "procedures_text": [[x["procedures_text"] for x in visits]],
    #     "drugs": [[[x["drugs"] for x in visits]]],
    #     "drugs_text": [[x["drugs_text"] for x in visits]],
    #     # "labevents": labevents,
    #     # "labevents_text": labevents_text
    #     "label": global_mortality_label,
    # }
   
   
    # For every condition in the sample (all visits). Record frequency.
    # Will be used to build code->index LUT.
    for code in sample['conditions']:
        CODE_COUNT[code] = CODE_COUNT.get(code, 0) + 1

    if SAMPLE_MULTIPLIER_:
        samples.extend(
            list(deepcopy(sample) for s in range(SAMPLE_MULTIPLIER_))
        )
        
    return samples

#### Test Load Readmission Dataset

In [ ]:
# set_task() returns a SampleEHRDataset object
READMISSION_PER_PATIENT_ICD_9_CODE_COUNT_ = {}
READMISSION_PER_PATIENT_ICD_9_CODE2IDX_ = {}
task_fn = functools.partial(readmission_pred_task_demb, READMISSION_PER_PATIENT_ICD_9_CODE_COUNT_)
readm_dataset = mimic3base.set_task(task_fn, task_name=readmission_pred_task_demb.__name__)
READMISSION_PER_PATIENT_ICD_9_CODE2IDX_ = {
    code: idx for idx, code in enumerate(sorted(READMISSION_PER_PATIENT_ICD_9_CODE_COUNT_.keys()))
}
readm_dataset.stat()
readm_dataset.samples[1]
# TODO(botelho3) could try a freq codes limit on this.
print(f"READMISSION_PER_PATIENT_ICD_9_CODE2IDX_ len: {len(READMISSION_PER_PATIENT_ICD_9_CODE2IDX_)}\n"
      f"{READMISSION_PER_PATIENT_ICD_9_CODE2IDX_}")
del readm_dataset

#### Test Load Mortality Dataset

In [ ]:
MORTALITY_PER_PATIENT_ICD_9_CODE_COUNT_ = {}
MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_ = {}
task_fn = functools.partial(mortality_pred_task_demb, MORTALITY_PER_PATIENT_ICD_9_CODE_COUNT_)
mor_dataset = mimic3base.set_task(task_fn, task_name=mortality_pred_task_demb.__name__)
MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_ = {
    code: idx for idx, code in enumerate(sorted(MORTALITY_PER_PATIENT_ICD_9_CODE_COUNT_.keys()))
}
mor_dataset.stat()
mor_dataset.samples[1]
# TODO(botelho3) could try a freq codes limit on this.
print(f"MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_ len: {len(MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_)}\n"
      f"{MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_}")
del mor_dataset

In [ ]:
# The original authors tackled 5 tasks
#   1. readmission
#   2. mortality
#   3. an ICU stay exceeding three days
#   4. an ICU stay exceeding seven days
#   5. diagnosis prediction

def readmission_pred_no_visit_task(patient, time_window=3):
    """
    patient is a <pyhealth.data.Patient> object
    """
    samples = []

    # we will drop the last visit
    for i in range(len(patient) - 1):
        visit: Visit = patient[i]
        next_visit: Visit = patient[i + 1]

        # step 1: define the readmission label 
        # get time difference between current visit and next visit
        time_diff = (next_visit.encounter_time - visit.encounter_time).days
        readmission_label = 1 if time_diff < time_window else 0
        
        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        drugs = visit.get_code_list(table="PRESCRIPTIONS")
        drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        
        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) == 0 * len(drugs_full) == 0:
            continue
        
        # step 3.5: build text lists from the ICD codes
        d_diag = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        d_proc = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        conditions_text = [d_diag.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [d_proc.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], d['dose'], d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
            
        # step 4: assemble the samples
        # TODO: should also exclude visit with age < 18
        samples.append(
            {
                "visit_id": visit.visit_id,
                "patient_id": patient.patient_id,
                # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
                "conditions": conditions,
                "procedures": procedures,
                "conditions_text": conditions_text,
                "procedures_text": procedures_text,
                "drugs": drugs,
                "drugs_text": drugs_text,
                # "labevents": labevents,
                # "labevents_text": labevents_text
                "label": readmission_label,
            }
        )
    # no cohort selection
    return samples


def mortality_pred_no_visit_task(patient):
    """
    patient is a <pyhealth.data.Patient> object
    """
    samples = []

    # loop over all visits but the last one
    for i in range(len(patient) - 1):

        # visit and next_visit are both <pyhealth.data.Visit> objects
        # there are no vists.attr_dict keys
        visit = patient[i]
        next_visit = patient[i + 1]
        # if i == 0: print(visit)

        # step 1: define the mortality_label
        if next_visit.discharge_status not in [0, 1]:
            mortality_label = 0
        else:
            mortality_label = int(next_visit.discharge_status)

        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="DIAGNOSES_ICD")
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        drugs = visit.get_code_list(table="PRESCRIPTIONS")
        drugs_full = visit.get_event_list(table="PRESCRIPTIONS")
        # if i == 0: print([d.attr_dict for d in drugs_full])
        # if i == 0: print(conditions)
        # if i == 0: print(procedures)
        # if i == 0: print(drugs)
        # TODO(botelho3) - add this datasource back in once we have full MIMIC-III dataset.
        # labevents = visit.get_code_list(table="LABEVENTS")

        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) == 0 * len(drugs_full) == 0: continue
        
        # step 3.5: build text lists from the ICD codes
        d_diag = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        d_proc = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        # if i == 0: print(d_diag)
        # if i == 0: print(d_proc)
        # Index 0 is shortname, index 1 is longname.
        # print([str(cond) + ' ' + str(d_diag.get(cond)) for cond in conditions])
        # print(d_proc.get(procedures[0]))
        conditions_text = [d_diag.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [d_proc.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], d['dose'], d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
        # TODO(botelho3) - add the labevents data source back in once we have full MIMIC-III dataset.
        # labevents_text =
        
        # step 4: assemble the samples
        samples.append(
            {
                "visit_id": visit.visit_id,
                "patient_id": patient.patient_id,
                # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
                "conditions": conditions,
                "procedures": procedures,
                "conditions_text": conditions_text,
                "procedures_text": procedures_text,
                "drugs": drugs,
                "drugs_text": drugs_text,
                # "labevents": labevents,
                # "labevents_text": labevents_text
                "label": mortality_label,
            }
        )
    return samples


# mimic3sample = mimic3base.set_task(task_fn=drug_recommendation_mimic3_fn) # use default task
# train_ds, val_ds, test_ds = split_by_patient(mimic3sample, [0.8, 0.1, 0.1])

### DataLoaders and Collate

#### Bert Collate

In [ ]:
def bert_per_patient_collate_function(data):
    """
    Collates a tensor of (batch_size, seq_len, bert_emb_len) i.e. (32, <events per patient>, 768)
    Need to pad the second dimension to max(<variable_per_patient>).
    
    TODO: Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
        sequences to the sample shape (max # visits, max # diagnosis codes). The padding infomation
        is stored in `mask`.
    
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (batch_size, max #conditions, max embedding size) of type torch.float
        masks: a tensor of shape (batch_size, max #conditions, max embedding_size) of type torch.bool
        rev_x: same as x but in reversed time. This will be used in our RNN model for masking 
        rev_masks: same as mask but in reversed time. This will be used in our RNN model for masking
        y: a tensor of shape (batch_size) of type torch.float
        
    Note that you can obtains the list of diagnosis codes and the list of hf labels
        using: `sequences, labels = zip(*data)`
    """
    # print(f"bert_per_patient_collate_function data[0] {data[0]}")
    tmp, labels = zip(*data)
    sequences = tmp 
    if not torch.is_tensor(tmp[0]):
        sequences = [torch.from_numpy(x) for x in tmp]
    
    # Quick stats on the amount of memory in each batch.
    sizes = [t.element_size() * t.nelement() for t in sequences]
    print(f'BertCollate tensor sizes\n'
          f'cum_size:{sum(sizes) / 1.0e6} MB\n'
          f'sizes:{sizes}\n'
          f'{[tuple(x.shape) for x in sequences]}\n')
    
    # Convert output labels for each sample in batch to tensor.
    # shape: (batch_size, 1)
    y = torch.tensor(labels, dtype=torch.float)
   
    num_patients = len(sequences)
    num_events = [patient.shape[0] for patient in sequences]
    embedding_length = [patient.shape[1] for patient in sequences]

    max_num_events = max(num_events)
    max_embedding_length = max(embedding_length)
    
    x = torch.zeros((num_patients, max_num_events, max_embedding_length), dtype=torch.float)
    rev_x = torch.zeros((num_patients, max_num_events, max_embedding_length), dtype=torch.float)
    # Mask dimensions are 1 less than inputs.
    masks = torch.zeros((num_patients, max_num_events), dtype=torch.bool)
    rev_masks = torch.zeros((num_patients, max_num_events), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):
        # Patient (#events, 768)
        j_visits = patient.shape[0]
        # for j_visit, visit in enumerate(patient):
        """
        TODO: update `x`, `rev_x`, `masks`, and `rev_masks`
        """
        # l = len(visit)
        x[i_patient, :j_visits, :] = patient[:, :].unsqueeze(0)
        # The tensor is (seq_length, emb_size). Leave embeddings,
        # flip temporal order of code/event sequence.
        rev_x[i_patient, :j_visits, :] = torch.flip(patient, dims=[0]).unsqueeze(0)
        masks[i_patient, :j_visits] = 1
        rev_masks[i_patient, :j_visits] = 1
      
        # TODO(botelho3) - comment this out to reduce spew.
        # if i_patient == 0:
        #     print(f"------ p: {i_patient} ------")
        #     print(x[i_patient, :, :25])
        #     print(rev_x[i_patient, :, :25])
        #     print(masks[i_patient, :, :25])
        #     print(rev_masks[i_patient, :, :25])
    
    return x, masks, rev_x, rev_masks, y

def bert_per_patient_collate_function_new_trainer(data):
    """
    Collates a tensor of (batch_size, seq_len, bert_emb_len) i.e. (32, <events per patient>, 768)
    Need to pad the second dimension to max(<variable_per_patient>).
    
    TODO: Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
        sequences to the sample shape (max # visits, max # diagnosis codes). The padding infomation
        is stored in `mask`.
    
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (batch_size, max #conditions, max embedding size) of type torch.float
        masks: a tensor of shape (batch_size, max #conditions, max embedding_size) of type torch.bool
        rev_x: same as x but in reversed time. This will be used in our RNN model for masking 
        rev_masks: same as mask but in reversed time. This will be used in our RNN model for masking
        y: a tensor of shape (batch_size) of type torch.float
        
    Note that you can obtains the list of diagnosis codes and the list of hf labels
        using: `sequences, labels = zip(*data)`
    """
    # print(f"bert_per_patient_collate_function data[0] {data[0]}")
    tmp, labels = zip(*data)
    sequences = None
    if not torch.is_tensor(tmp[0]):
        sequences = [torch.from_numpy(x) for x in tmp]
    
    # Quick stats on the amount of memory in each batch.
    sizes = [t.element_size() * t.nelement() for t in sequences]
    
    # Convert output labels for each sample in batch to tensor.
    # shape: (batch_size, 1)
    y = torch.tensor(labels, dtype=torch.float)
   
    num_patients = len(sequences)
    num_events = [patient.shape[0] for patient in sequences]
    embedding_length = [patient.shape[1] for patient in sequences]

    max_num_events = max(num_events)
    max_embedding_length = max(embedding_length)
    
    x = torch.zeros((num_patients, max_num_events, max_embedding_length), dtype=torch.float)
    rev_x = torch.zeros((num_patients, max_num_events, max_embedding_length), dtype=torch.float)
    # Mask dimensions are 1 less than inputs.
    masks = torch.zeros((num_patients, max_num_events), dtype=torch.bool)
    rev_masks = torch.zeros((num_patients, max_num_events), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):
        # Patient (#events, 768)
        j_visits = patient.shape[0]
        # for j_visit, visit in enumerate(patient):
        """
        TODO: update `x`, `rev_x`, `masks`, and `rev_masks`
        """
        # l = len(visit)
        x[i_patient, :j_visits, :] = patient[:, :].unsqueeze(0)
        # The tensor is (seq_length, emb_size). Leave embeddings,
        # flip temporal order of code/event sequence.
        rev_x[i_patient, :j_visits, :] = torch.flip(patient, dims=[0]).unsqueeze(0)
        masks[i_patient, :j_visits] = 1
        rev_masks[i_patient, :j_visits] = 1
      
        # TODO(botelho3) - comment this out to reduce spew.
        # if i_patient == 0:
        #     print(f"------ p: {i_patient} ------")
        #     print(x[i_patient, :, :25])
        #     print(rev_x[i_patient, :, :25])
        #     print(masks[i_patient, :, :25])
        #     print(rev_masks[i_patient, :, :25])
    return {
        'x': x,
        'masks': masks,
        'rev_x': rev_x,
        'rev_masks': rev_masks,
        'y': y,
    }


#### Bert Load

In [ ]:
# Could use DistilBERT to decrease embedding time.
# https://huggingface.co/docs/transformers/model_doc/distilbert?highlight=distilberttokenizerfast#distilbert

class BertTextEmbedTransform(object):
    """Transform a sample's (a single visit's) text into 1 embedding vector.
    
    The embeddings of each text field are combined by embedding
    each separately then summing.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """
    # https://stackoverflow.com/questions/69517460/bert-get-sentence-embedding
    # https://huggingface.co/docs/transformers/internal/tokenization_utils#transformers.tokenization_utils_base.PreTrainedTokenizerBase
    # https://gmihaila.github.io/tutorial_notebooks/bert_inner_workings/

    def __init__(self, bert_model: Any, embedding_size: int, use_tokenizer_fast: bool):
        assert isinstance(embedding_size, (int, tuple))
        self.use_gpu = BERT_USE_GPU_
        self.cuda = GPU_STR_
        self.cache_dir = os.path.join(os.getcwd(), CACHE_DIR_)
        self.bert_config = BertConfig()
        # self.bert_config = AutoConfig.from_pretrained("bert-base-uncased")
        # self.bert_model = BertModel(self.bert_config).from_pretrained('bert-base-uncased', config=self.bert_config)
        # self.bert_model = AutoModel.from_pretrained('bert-base-uncased')
        with torch.no_grad():
            self.bert_model = BertModel(self.bert_config).from_pretrained('bert-base-uncased')
        if self.use_gpu:
            self.bert_model.to(self.cuda)
        self.bert_config = self.bert_model.config
        self.bert_model.eval()
        # We unfortunately can't put the tokenizer on the GPU.
        # https://stackoverflow.com/questions/66096703/running-huggingface-bert-tokenizer-on-gpu
        if use_tokenizer_fast:
            self.tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', model_max_length=100)
        else:
            self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', model_max_length=100)
    
    def _tokenize_text(self, text: str) -> str:
        tokenized_text = self.tokenizer.tokenize(text)
        return tokenized_text
    
    def _get_embeddings_of_sentences_with_mask(self, field, pad) -> torch.tensor:
        return self._get_embeddings_of_sentences(field[:pad])
        
    
    def _get_embeddings_of_sentences(self, sentences: List[str]) -> torch.tensor:
        # tokenized_sentences = [self.tokenizer.tokenize(t, padding=True) for t in sentences]
        # Tokenize the input sentence with attention masks.
        batch_enc = self.tokenizer.batch_encode_plus(sentences, padding=True,
                                return_attention_mask=True, return_length=True)
        # print(f'input sentence: {sentences[0]}\n'
        #       f'token sentence: {self.tokenizer.decode(batch_enc["input_ids"][0])}\n')
        batch_enc_tensor = batch_enc.convert_to_tensors(tensor_type=TensorType.PYTORCH)
        if self.use_gpu:
            batch_enc_tensor.to(self.cuda)
      
        # Run BERT model forward pass on tokenized input.
        with torch.no_grad():
            embeddings = self.bert_model(input_ids=batch_enc_tensor['input_ids'],
                                         attention_mask=batch_enc_tensor['attention_mask'],
                                         token_type_ids=batch_enc_tensor['token_type_ids'],
                                         # could turn off so we're faster
                                         output_attentions=True)
        # embeddings, _ = self.bert_model(**batch_enc)
        # print(f'embeddings:\n {dir(embeddings)}')
        #attention = encoded['attention_mask'].reshape((lhs.size()[0], lhs.size()[1], -1)).expand(-1, -1, 768)
       
        # These may be on GPU.
        return embeddings.last_hidden_state, embeddings.attentions 
    
    def __call__(self, sample):
        len_embeddings = (len(sample['conditions_text']) +
                          len(sample['procedures_text']) +
                          len(sample['drugs_text']))
      
        sample_text = []
        sample_masks = []
        if sample.get('conditions_text_pad'):
            pad = sample['conditions_text_pad']
            sample_text.extend(sample['conditions_text'][:pad])
        if sample.get('procedures_text_pad'):
            pad = sample['procedures_text_pad']
            sample_text.extend(sample['procedures_text'][:pad])
        if sample.get('drugs_text_pad'):
            pad = sample['drugs_text_pad']
            sample_text.extend(sample['drugs_text'][:pad])
           
        sample_embeddings, sample_attentions = self._get_embeddings_of_sentences(sample_text)
        
        # https://stackoverflow.com/questions/61323621/how-to-understand-hidden-states-of-the-returns-in-bertmodelhuggingface-transfo
        # https://datascience.stackexchange.com/questions/66207/what-is-purpose-of-the-cls-token-and-why-is-its-encoding-output-important
        # Take only the last hidden state embeddings from BERT.
        # We need to take index 0, because it's the CLS token prepended to our sentence.
        # The [CLS] represents the meaning of the whole sentence.
        # embeddings = torch.squeeze(sample_embeddings[:, -1, :], dim=1)
        embeddings = torch.squeeze(sample_embeddings[:, 0, :], dim=1)
        if self.use_gpu:
            assert(embeddings.device == torch.device('cuda:0'))
        else:
            assert(embeddings.device == torch.device('cpu'))
        if self.use_gpu:
            embeddings = embeddings.to('cpu')
        
        # We could multiply by attentions here:
        # attentions = torch.squeeze(sample_attentions[:, -1, :], dim=1)
        # embeddings = embeddings * attentions

        # The 1st dimension is seq length. The second dimension is embedding length of each sentence.
        assert(embeddings.shape[-1] == self.bert_config.hidden_size)
        assert(len(embeddings.shape) == 2)
        assert(embeddings.dtype == torch.float)
        
        # Return the ((model_inputs), label) 
        return (embeddings, sample['label'])
    
    
class TextEmbedDataset(SampleDataset):
    '''The BERT text embedding process is very slow. We want to avoid it.
   
    To prevent re-processing of the same input cache the sample locally.
    Some suggestions here:
        https://stackoverflow.com/questions/61393613/pytorch-speed-up-data-loading.
        https://discuss.pytorch.org/t/best-practice-to-cache-the-entire-dataset-during-first-epoch/19608
        
    1. Preprocess and write the preprocessed text back out to disk.
    2. Cache the transform output in a hashtable. See functools.lru_cache().
    3. https://pytorch.org/data/main/ ?
    
    Some concerns related to num_workers > 1, i.e. multiprocessing enabled.
    See torch.save() to cache a tensor.
    '''
    
    def __init__(self, dataset: SampleDataset, transform=None, should_cache=True):
        """Wraps a SampleEHRDataset with transforms.
        Arguments:
            dataset: dataset to transform
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        # self.dataset = dataset
        self.transform = transform
        self.transformed = [False for x in dataset]
        self.should_cache = should_cache
        super().__init__([x for x in dataset])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            assert(False)
            idx = idx.tolist()

        # Cache the transformed version of the data.
        sample = None
        if self.should_cache and self.transform and not self.transformed[idx]:
            self.samples[idx] = self.transform(self.samples[idx])
            self.transformed[idx] = True
            sample = self.samples[idx]
        elif not self.should_cache:
            sample = self.transform(self.samples[idx])

        return sample

In [ ]:
gc.collect()

# Create the transform that will take each sample (visit) in the dataset
# and convert the text description of the visit into a single embedding.
bert_xform = BertTextEmbedTransform(None, EMBEDDING_DIM_, use_tokenizer_fast=True)
BERT_EMBEDDING_SIZE = bert_xform.bert_config.hidden_size

# Mortality Task Datasets: set_task -> SampleEHRDataset(SampleBaseDataset)
# Add a transform to convert the visit into an embedding.
MORT_DEMB_CODE2IDX = {}
MORT_DEMB_CODE_COUNT = {}
task_fn = functools.partial(mortality_pred_task_demb, MORT_DEMB_CODE_COUNT)
mortality_demb_dataset = mimic3base.set_task(task_fn, task_name=mortality_pred_task_demb.__name__)
MORT_DEMB_CODE2IDX = {
    code: idx for idx, code in enumerate(sorted(MORT_DEMB_CODE_COUNT.keys()))
}
print(f"MORT_DEMB_CODE2IDX {len(MORT_DEMB_CODE2IDX)}")
# Wrap the default pyHealth dataset class in our own wrapper. The wrapper takes each
# sample and applies BERT to xform text->pytorch.tensor.
mortality_demb_dataset = TextEmbedDataset(mortality_demb_dataset, transform=bert_xform)


# mort_demb_train_ds, mort_demb_val_ds, mort_demb_test_ds = split_by_patient(mortality_demb_dataset, [0.8, 0.1, 0.1])

# # create dataloaders (torch.data.DataLoader)
# # mort_train_loader = get_dataloader(train_ds, batch_size=32, shuffle=True, collate_fn)
# # mort_val_loader = get_dataloader(val_ds, batch_size=32, shuffle=False)
# # mort_test_loader = get_dataloader(test_ds, batch_size=32, shuffle=False)

# mort_demb_train_loader = DataLoader(
#     mort_demb_train_ds,
#     batch_size=BATCH_SIZE_,
#     shuffle=SHUFFLE_,
#     collate_fn=bert_per_patient_collate_function,
# )
# mort_demb_val_loader = DataLoader(
#     mort_demb_val_ds,
#     batch_size=BATCH_SIZE_,
#     shuffle=SHUFFLE_,
#     collate_fn=bert_per_patient_collate_function,
# )
# mort_demb_test_loader = DataLoader(
#     mort_demb_test_ds,
#     batch_size=BATCH_SIZE_,
#     shuffle=SHUFFLE_,
#     collate_fn=bert_per_patient_collate_function,
# )


In [ ]:
gc.collect()

# Create the transform that will take each sample (visit) in the dataset
# and convert the text description of the visit into a single embedding.
bert_xform = BertTextEmbedTransform(None, EMBEDDING_DIM_, use_tokenizer_fast=True)
BERT_EMBEDDING_SIZE = bert_xform.bert_config.hidden_size

# Readmission Task Datasets: set_task -> SampleEHRDataset(SampleBaseDataset)
# Add a transform to convert the visit into an embedding.
READM_DEMB_CODE2IDX = {}
READM_DEMB_CODE_COUNT = {}
task_fn = functools.partial(readmission_pred_task_demb, READM_DEMB_CODE_COUNT)
readmission_demb_dataset = mimic3base.set_task(task_fn, task_name=readmission_pred_task_demb.__name__)
READM_DEMB_CODE2IDX = {
    code: idx for idx, code in enumerate(sorted(READM_DEMB_CODE_COUNT.keys()))
}
print(f"READM_DEMB_CODE2IDX {len(READM_DEMB_CODE2IDX)}")
# Wrap the default pyHealth dataset class in our own wrapper. The wrapper takes each
# sample and applies BERT to xform text->pytorch.tensor.
readmission_demb_dataset = TextEmbedDataset(readmission_demb_dataset, transform=bert_xform)

In [ ]:
if False:
    # Verify DataLoader properties.
    # Quick test without running the whole RNN training process.

    # from torch.utils.data import DataLoader

    # loader = DataLoader(mort_, batch_size=10, collate_fn=collate_fn)
    loader_iter = iter(mort_demb_train_loader)
    # for _ in loader_iter:
    #     pass
    try:
        x, masks, rev_x, rev_masks, y = next(loader_iter)
    except StopIteration as e:
        print(e)

    assert x.dtype == torch.float
    assert rev_x.dtype == torch.float
    assert y.dtype == torch.float
    assert masks.dtype == torch.bool
    assert rev_masks.dtype == torch.bool

    assert x.shape == (BATCH_SIZE_, 3, 105)
    assert y.shape == (BATCH_SIZE_, 1)
    assert masks.shape == (BATCH_SIZE_, 10, 3)

    # assert x[0][0].sum() == 9
    # assert masks[0].sum() == 2

#### CodeEmb Collate 

In [ ]:
def code_emb_per_visit_collate_function(code2idx: Dict[str, int], data: List[Any]) -> Tuple[Any]:
    """
    TODO: Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
    sequences to the sample shape (max # visits, max # diagnosis codes). The padding infomation
    is stored in `mask`.
                                   
    
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
    x: a tensor of shape (# patients, max # visits, max # diagnosis codes) of type torch.long
    masks: a tensor of shape (# patients, max # visits, max # diagnosis codes) of type torch.bool
    rev_x: same as x but in reversed time. This will be used in our RNN model for masking
    rev_masks: same as mask but in reversed time. This will be used in our RNN model for masking
    y: a tensor of shape (# patiens) of type torch.float
        
    Note that you can obtains the list of diagnosis codes and the list of hf labels\n",
    using: `sequences, labels = zip(*data)`\n",
    """
    
    samples = data
    
    y = torch.tensor([s['label'] for s in samples], dtype=torch.float)
    
    num_patients = len(samples)
    num_visits = [patient['num_visits'] for patient in samples]
    num_codes = []
    for patient_idx, _ in enumerate(num_visits):
        num_codes.extend([len(visit) for visit in samples[patient_idx]['conditions']])
        
    # print(f'num samples: {len(samples)}')
    # print(f'num visits: {num_visits}')
    # print(f'num codes: {num_codes}')
    # print(f'code_emb_per_patient_collate_function data[0]:\n{data[0]}')
 
    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
    # max_num_codes = len(MORTALITY_PER_VISIT_ICD_9_CODE2IDX_)
    assert(max_num_codes > 0)
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    rev_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    for i_patient, patient in enumerate(samples):
        nvisits = patient['num_visits']
        for j_visit in range(nvisits):
            """
            TODO: update `x`, `rev_x`, `masks`, and `rev_masks`
            """
            codes = patient['conditions'][j_visit] 
            indices = [code2idx[code] for code in codes]
            l = len(codes)
            if len(indices) < 1:
                print(f'No code indices after lookup')
                print(f'codes {codes}'+'\n'+f'indices{indices}')
                print(f'patient\n{patient}')
                assert(len(indices) >= 1)
            # for idx in indices:
            x[i_patient, j_visit, 0:l] = torch.tensor(indices)
            rev_x[i_patient, nvisits-1-j_visit, 0:l] = torch.tensor(indices)
            masks[i_patient, j_visit, 0:l] = 1
            rev_masks[i_patient, nvisits-1-j_visit, 0:l] = 1
        # if i_patient == 0:
        #     print(f"------ code_emb_per_visit_collate_function p: {i_patient} ------")
        #     print(x[i_patient, :, ])
        #     print(rev_x[i_patient, :, ])
        #     print(masks[i_patient, :, ])
        #     print(rev_masks[i_patient, :, ])
    
    return x, masks, rev_x, rev_masks, y

#### CodeEmb Load

In [ ]:
# Mortality Task Datasets: set_task -> SampleEHRDataset(SampleBaseDataset)
MORT_CEMB_CODE2IDX = {}
MORT_CEMB_CODE_COUNT = {}
mortality_cemb_ds = mimic3base.set_task(
    task_fn=functools.partial(mortality_pred_task_cemb, MORT_CEMB_CODE_COUNT),
    task_name=mortality_pred_task_cemb.__name__)
# The set_task(...) function iterates over all samples.
# Applying the task to each before returning a new dataset.
# Since all samples have been processed we have observed all codes and can
# build the code->index LUT.
MORT_CEMB_CODE2IDX = {
    code: idx for idx, code in enumerate(sorted(set(MORT_CEMB_CODE_COUNT.keys())))
}
print(f'MORT_CEMB_CODE2IDX {len(MORT_CEMB_CODE2IDX)}')

# # We need to provide the code->index LUT to the collate function.
wrap_code_emb_per_visit_collate_function = functools.partial(
    code_emb_per_visit_collate_function,
    MORT_CEMB_CODE2IDX)

# # Split the dataset into train, val, and test.
# mort_cemb_train_ds, mort_cemb_val_ds, mort_cemb_test_ds = split_by_patient(mortality_cemb_ds, [0.8, 0.1, 0.1])
# mort_cemb_train_loader = DataLoader(
#     mort_cemb_train_ds,
#     batch_size=BATCH_SIZE_,
#     shuffle=SHUFFLE_,
#     collate_fn=wrap_code_emb_per_visit_collate_function,
# )
# mort_cemb_val_loader = DataLoader(
#     mort_cemb_val_ds,
#     batch_size=BATCH_SIZE_,
#     shuffle=SHUFFLE_,
#     collate_fn=wrap_code_emb_per_visit_collate_function,
# )
# mort_cemb_test_loader = DataLoader(
#     mort_cemb_test_ds,
#     batch_size=BATCH_SIZE_,
#     shuffle=SHUFFLE_,
#     collate_fn=wrap_code_emb_per_visit_collate_function,
# )


In [ ]:
# Mortality Task Datasets: set_task -> SampleEHRDataset(SampleBaseDataset)
READM_CEMB_CODE2IDX = {}
READM_CEMB_CODE_COUNT = {}
readm_cemb_ds = mimic3base.set_task(
    task_fn=functools.partial(readmission_pred_task_cemb, READM_CEMB_CODE_COUNT),
    task_name=readmission_pred_task_cemb.__name__)
# The set_task(...) function iterates over all samples.
# Applying the task to each before returning a new dataset.
# Since all samples have been processed we have observed all codes and can
# build the code->index LUT.
READM_CEMB_CODE2IDX = {
    code: idx for idx, code in enumerate(sorted(set(READM_CEMB_CODE_COUNT.keys())))
}
print(f'READM_CEMB_CODE2IDX {len(READM_CEMB_CODE2IDX)}')

In [ ]:
if False:
    # Verify DataLoader properties.
    # Quick test without running the whole RNN training process.

    # from torch.utils.data import DataLoader

    # loader = DataLoader(mort_, batch_size=10, collate_fn=collate_fn)
    loader_iter = iter(mort_cemb_train_loader)
    for _ in loader_iter:
        pass

    try:
        x, masks, rev_x, rev_masks, y = next(loader_iter)
    except StopIteration as e:
        print(e)

    # assert x.dtype == torch.float
    # assert rev_x.dtype == torch.float
    # assert y.dtype == torch.float
    # assert masks.dtype == torch.bool
    # assert rev_masks.dtype == torch.bool

    # assert x.shape == (BATCH_SIZE_, 3, 105)
    # assert y.shape == (BATCH_SIZE_, 1)

#### Dataset Caching

In [ ]:

# The collate function idea doesn't work because collating requires the batch size.
# The batch size can change so it doesn't really work.
# Unless we can use https://pytorch.org/docs/stable/data.html#disable-automatic-batching
# But then the train/val/test datasets & batch sizes are fixed after the initial save to file.
# Should probably incorporate the batch size into the file hash.
# That would also mean we should return 


# class SimpleDataset(Dataset):
#     '''Wraps a list. For post cache datasets.
#     '''
#     def __init__(self, samples: Iterable):
#         self.samples = samples
        
#     def __len__(self):
#         return len(self.samples)
    
#     def __getitem__(self, index):
#         return self.samples[index]
    

# class DatasetCacher:
#     '''Caches a dataset to file. Returns dataset from file with loader if desired.
#         Datasets are defined by 3 params:
#             1. name
#             2. batch_size
#             3. length
#     '''
#     def __init__(self):
#         pass
   

#     def _HashStr(self, s) -> str:
#         return hashlib.md5(s.encode()).hexdigest() 


#     def _MakeMetadata(self, name: str, batch_size: int, length: int) -> Dict:
#         metadata = {
#             'auto_batch': batch_size == 0,
#             'batch_size': batch_size,
#             'length': length,
#             'dataset_name': name
#         }
#         return metadata

    
#     def _GetHashString(self, args) -> str:
#         '''Compute filename for cache.
#         '''
#         args_to_hash = [
#             args['dataset_name'],
#             args['auto_batch'],
#             args['batch_size'],
#             args['length']
#         ]
#         filename = '_'.join([
#             self._HashStr("+".join([str(arg) for arg in args_to_hash])),
#             args['dataset_name'],
#             'bs_' + str(args['batch_size']),
#             'len_' + str(args['length']),
#             '.pkl',
#         ])
#         return os.path.join(MODULE_CACHE_PATH, filename)

    
#     def DatasetToCacheFromLoader(self, loader: DataLoader,
#                                  task_fn: Callable,
#                                  batch_size: int=0,
#                                  overwrite: bool=False,
#                                  extra_data: Dict=None):
#         to_pickle = {
#             'dataset': [x for x in loader],
#             'metadata': self._MakeMetadata(task_fn.__name__, batch_size, len(loader.dataset))
#         }
#         if extra_data:
#             to_pickle['extra_data'] = extra_data
#         fname = self._GetHashString(to_pickle['metadata'])
#         logging.info(f'Caching dataset {to_pickle["metadata"]} to file `{fname}`.')
#         if not os.path.exists(fname) or overwrite:
#             save_pickle(to_pickle, fname)
#         return to_pickle['metadata']
        
    
#     def DatasetToCache(self, dataset: Dataset,
#                            task_fn: Callable,
#                            batch_size: int=0,
#                            overwrite: bool=False,
#                            extra_data: Dict=None) -> None:
#         '''Write all items in the dataset to file.
        
#           If the dataset already exists on disk this function is a no-op.
#           The dataset is hashed by iterating over the whole dataset and writing
#           all items to disk using pickle.
#         '''
#         to_pickle = {
#             'dataset': [x for x in dataset],
#             'metadata': self._MakeMetadata(task_fn.__name__, batch_size, len(dataset))
#         }
#         if extra_data:
#             to_pickle['extra_data'] = extra_data
#         fname = self._GetHashString(to_pickle['metadata'])
#         logging.info(f'Caching dataset {to_pickle["metadata"]} to file `{fname}`.')
#         if not os.path.exists(fname) or overwrite:
#             save_pickle(to_pickle, fname)
#         return to_pickle['metadata']
    
    
#     def DatasetFromCache(self, task_fn: Callable,
#                          batch_size: int,
#                          length: int) -> Dataset:
#         '''Retrieve all items in the dataset from file.
#         '''
#         metadata = self._MakeMetadata(task_fn.__name__, batch_size, length)
#         fname = self._GetHashString(metadata)
#         logging.info(f'Loading cached dataset {metadata} from file `{fname}`.')
#         d = load_pickle(fname)
#         return SimpleDataset(d['dataset']), d['metadata'] 
   

#     def DataloaderFromCache(self, *args, **kwargs) -> DataLoader:
#         dataset, metadata = self.DatasetFromCache(*args, **kwargs)
#         batch_size = metadata['batch_size']
#         # Disable auto batch if batch_size=0. Good for already collated data.
#         loader = DataLoader(dataset,
#                    batch_size=None if batch_size < 2 else batch_size,
#                    shuffle=False)
#         return loader, metadata
    

##### Cemb

In [ ]:
# In order to cut down the data size the collate fn is going to have to happen later during batch run.
cacher = DatasetCacher()

wrap_code_emb_per_visit_collate_function = functools.partial(
    code_emb_per_visit_collate_function,
    MORT_CEMB_CODE2IDX)
mort_cemb_loader = DataLoader(
    mortality_cemb_ds,
    batch_size=BATCH_SIZE_,
    shuffle=SHUFFLE_,
    collate_fn=wrap_code_emb_per_visit_collate_function,
)
extra_data = {
    'code2idx': MORT_CEMB_CODE2IDX,
    'embed_index_size': len(MORT_CEMB_CODE2IDX),
    'keywords': ['x', 'masks', 'rev_x', 'rev_masks', 'y'],
}
mort_cemb_cacher_metadata = cacher.DatasetToCacheFromLoader(mort_cemb_loader,
                      mortality_pred_task_cemb,
                      batch_size=0,  # we already batched using loader+collate.
                      overwrite=False,
                      extra_data=extra_data)
# i = iter(mort_cemb_loader)
# print(next(i))

In [ ]:
print(f"len dataset {len(mort_cemb_loader.dataset)}")
cacher = DatasetCacher()
metadata_in = {'batch_size': 0, 'length': len(mort_cemb_loader.dataset)}
mort_cemb_loader, mort_cemb_metadata = (
    cacher.DataloaderFromCache(mortality_pred_task_cemb.__name__,
                               metadata_in['batch_size'],
                               metadata_in['length'])
)
i = iter(mort_cemb_loader)
print(next(i))

In [ ]:
# In order to cut down the data size the collate fn is going to have to happen later during batch run.
cacher = DatasetCacher()

wrap_code_emb_per_visit_collate_function = functools.partial(
    code_emb_per_visit_collate_function,
    READM_CEMB_CODE2IDX)
readm_cemb_loader = DataLoader(
    readm_cemb_ds,
    batch_size=BATCH_SIZE_,
    shuffle=SHUFFLE_,
    collate_fn=wrap_code_emb_per_visit_collate_function,
)
extra_data = {
    'code2idx': READM_CEMB_CODE2IDX,
    'embed_index_size': len(READM_CEMB_CODE2IDX),
    'keywords': ['x', 'masks', 'rev_x', 'rev_masks', 'y'],
}
readm_cemb_cacher_metadata = cacher.DatasetToCacheFromLoader(readm_cemb_loader,
                      readmission_pred_task_cemb,
                      batch_size=0,  # we already batched using loader+collate.
                      overwrite=False,
                      extra_data=extra_data)
# i = iter(mort_cemb_loader)
# print(next(i))

In [ ]:
print(f"len dataset {len(readm_cemb_loader.dataset)}")
cacher = DatasetCacher()
metadata_in = {'batch_size': 0, 'length': len(readm_cemb_loader.dataset)}
readm_cemb_loader, readm_cemb_metadata = (
    cacher.DataloaderFromCache(readmission_pred_task_cemb.__name__,
                               metadata_in['batch_size'],
                               metadata_in['length'])
)
i = iter(readm_cemb_loader)
print(next(i))

In [ ]:
if False:
    del readm_cemb_ds
    del readm_cemb_loader
    del mortality_cemb_ds
    del mortality_cemb_loader

##### Demb 

In [ ]:
# Might need packed_sequence for this.
cacher = DatasetCacher()

# indices = list(range(0, min(2000, len(mortality_demb_dataset)) ))
indices = list(range(0, min(10000, len(mortality_demb_dataset)) ))
print(len(mortality_demb_dataset))
print(len(torch.utils.data.Subset(mortality_demb_dataset, indices)))
# mort_demb_loader = DataLoader(
#     torch.utils.data.Subset(mort_demb_train_ds, indices),
#     batch_size=BATCH_SIZE_,
#     shuffle=SHUFFLE_,
#     collate_fn=bert_per_patient_collate_function,
# )
extra_data = {
    'embed_index_size': 768,
    'keywords': ['x', 'masks', 'rev_x', 'rev_masks', 'y'],
}

def numpy_conversion_function(sample):
    # This is faster than pickling tensors
    # https://github.com/pytorch/pytorch/issues/9168
    tensor, label = sample[0]
    return (tensor.numpy(), label)

def bytes_conversion_function(sample):
    # This is faster than pickling tensors
    # https://github.com/pytorch/pytorch/issues/9168
    tensor, label = sample[0]
    bio = io.BytesIO()
    torch.save(tensor, bio)
    return (bio.getvalue(), label)
    
# mort_demb_cacher_metadata = cacher.DatasetToCacheFromLoader(mort_demb_loader,
#                       mortality_pred_task_demb,
#                       batch_size=0,  # we already batched using loader+collate.
#                       overwrite=True,
#                       extra_data=extra_data)
mort_demb_loader = DataLoader(
    torch.utils.data.Subset(mortality_demb_dataset, indices),
    batch_size=1,
    shuffle=SHUFFLE_,
    collate_fn=numpy_conversion_function,
    # collate_fn=bytes_conversion_function,
)
mort_demb_cacher_metadata = cacher.DatasetToCacheFromLoader(mort_demb_loader,
                      mortality_pred_task_demb,
                      batch_size=0,  # we already batched using loader+collate.
                      overwrite=False,
                      extra_data=extra_data)
# i = iter(mort_demb_loader)
# print(next(i))

In [ ]:
print(f"len dataset {len(mort_demb_loader.dataset)}")
cacher = DatasetCacher()
metadata_in = {'batch_size': 0, 'length': len(mort_demb_loader.dataset)}
mort_demb_loader, mort_demb_metadata = (
    cacher.DataloaderFromCache(mortality_pred_task_demb.__name__,
                               metadata_in['batch_size'],
                               metadata_in['length'])
)
i = iter(mort_demb_loader)
print(next(i))

In [ ]:
# Might need packed_sequence for this.
cacher = DatasetCacher()

# indices = list(range(0, min(2000, len(readmission_demb_dataset)) ))
indices = list(range(0, min(10000, len(readmission_demb_dataset)) ))
print(len(readmission_demb_dataset))
print(len(torch.utils.data.Subset(readmission_demb_dataset, indices)))
extra_data = {
    'embed_index_size': 768,
    'keywords': ['x', 'masks', 'rev_x', 'rev_masks', 'y'],
}

def numpy_conversion_function(sample):
    # This is faster than pickling tensors
    # https://github.com/pytorch/pytorch/issues/9168
    tensor, label = sample[0]
    return (tensor.numpy(), label)

def bytes_conversion_function(sample):
    # This is faster than pickling tensors
    # https://github.com/pytorch/pytorch/issues/9168
    tensor, label = sample[0]
    bio = io.BytesIO()
    torch.save(tensor, bio)
    return (bio.getvalue(), label)
    
# mort_demb_cacher_metadata = cacher.DatasetToCacheFromLoader(mort_demb_loader,
#                       mortality_pred_task_demb,
#                       batch_size=0,  # we already batched using loader+collate.
#                       overwrite=True,
#                       extra_data=extra_data)
readm_demb_loader = DataLoader(
    torch.utils.data.Subset(readmission_demb_dataset, indices),
    batch_size=1,
    shuffle=SHUFFLE_,
    collate_fn=numpy_conversion_function,
    # collate_fn=bytes_conversion_function,
)
readm_demb_cacher_metadata = cacher.DatasetToCacheFromLoader(readm_demb_loader,
                      readmission_pred_task_demb,
                      batch_size=0,  # we already batched using loader+collate.
                      overwrite=False,
                      extra_data=extra_data)
# i = iter(mort_demb_loader)
# print(next(i))

In [ ]:
print(f"len dataset {len(readm_demb_loader.dataset)}")
cacher = DatasetCacher()
metadata_in = {'batch_size': 0, 'length': len(readm_demb_loader.dataset)}
readm_demb_loader, readm_demb_metadata = (
    cacher.DataloaderFromCache(readmission_pred_task_demb.__name__,
                               metadata_in['batch_size'],
                               metadata_in['length'])
)
i = iter(readm_demb_loader)
print(next(i))

In [ ]:
if False:
    del bert_xform
    del mort_demb_loader
    del mortality_demb_dataset
    del readm_demb_loader
    del readmission_demb_dataset

### Bert Fine Tune Collate

In [ ]:
class BertFineTuneTransform(object):
    """Transform a sample's (a single visit's) text into 1 embedding vector.
    
    The embeddings of each text field are combined by embedding
    each separately then summing.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """
    def __init__(self):
        self.tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', model_max_length=100)
        self.tokenzier = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
        print(f'Tokenizer: {self.tokenizer}')
        # max length of sentence not real emb size.
        self.emb_size = 20  # 15 worked here.
    
    def _get_embeddings_of_sentences_with_mask(self, field, pad) -> torch.tensor:
        return self._get_embeddings_of_sentences(field[:pad])
        
    def _get_embeddings_of_sentences(self, sentences: List[str]) -> torch.tensor:
        batch_enc = self.tokenizer.batch_encode_plus(sentences, padding=True,
                                return_attention_mask=True, return_length=True,
                                truncation='longest_first', max_length=self.emb_size)
        batch_enc_tensor = batch_enc.convert_to_tensors(tensor_type=TensorType.PYTORCH)
        # decoded = self.tokenizer.decode(batch_enc['input_ids'][0])
        # print(f'decoded {decoded}')
        # assert(decoded[0:5] == '[CLS]')
        return batch_enc_tensor
    
    def __call__(self, sample):
        len_embeddings = (len(sample['conditions_text']) +
                          len(sample['procedures_text']) +
                          len(sample['drugs_text']))
        sample_text = []
        sample_masks = []
        if sample.get('conditions_text_pad'):
            pad = sample['conditions_text_pad']
            sample_text.extend(sample['conditions_text'][:pad])
        if sample.get('procedures_text_pad'):
            pad = sample['procedures_text_pad']
            sample_text.extend(sample['procedures_text'][:pad])
        if sample.get('drugs_text_pad'):
            pad = sample['drugs_text_pad']
            sample_text.extend(sample['drugs_text'][:pad])
            
        batch_enc_tensor = self._get_embeddings_of_sentences(sample_text)
        assert(batch_enc_tensor['input_ids'].device == torch.device('cpu'))
        assert(batch_enc_tensor['token_type_ids'].device == torch.device('cpu'))
        # print(f'FT transform shape\n'
        #       f"iids {batch_enc_tensor['input_ids'].shape}\n"
        #       f"ttids {batch_enc_tensor['token_type_ids'].shape}")
        # Return the ((model_inputs), label) 
        d = {
            'input_ids': batch_enc_tensor['input_ids'],
            'attention_masks': batch_enc_tensor['attention_mask'],
            'token_type_ids': batch_enc_tensor['token_type_ids'],
        }
        return (d, sample['label'])

In [ ]:
def bert_fine_tune_collate(data):
    """
    Collates a tensor of (batch_size, seq_len, bert_emb_len) i.e. (32, <events per patient>, 768)
    Need to pad the second dimension to max(<variable_per_patient>).
    
    TODO: Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
        sequences to the sample shape (max # visits, max # diagnosis codes). The padding infomation
        is stored in `mask`.
    
    Arguments:
        data: list of ({'inputs_ids':tensor, 'attention_mask':tensor, 'token_type_ids':tensor}, y)
        
    Outputs:
        x: a tensor of shape (batch_size, max #conditions, max embedding size) of type torch.float
        masks: a tensor of shape (batch_size, max #conditions, max embedding_size) of type torch.bool
        rev_x: same as x but in reversed time. This will be used in our RNN model for masking 
        rev_masks: same as mask but in reversed time. This will be used in our RNN model for masking
        y: a tensor of shape (batch_size) of type torch.float
        
    Note that you can obtains the list of diagnosis codes and the list of hf labels
        using: `sequences, labels = zip(*data)`
    """
    kMaxEvents = 25
    kMaxWordLength = 20
    sequences, labels = zip(*data)
    
    # Convert output labels for each sample in batch to tensor.
    # shape: (batch_size, 1)
    y = torch.tensor(labels, dtype=torch.float)
   
    num_patients = len(sequences)
    num_events = [patient['input_ids'].shape[0] for patient in sequences]
    embedding_length = [patient['input_ids'].shape[1] for patient in sequences]
    # if len(sequences) > 1:
    #     assert(sequences[0]['input_ids'].shape[1] == sequences[1]['input_ids'].shape[1])

    max_num_events = min(kMaxEvents, max(num_events))
    max_embedding_length = min(kMaxWordLength, max(embedding_length))
    
    input_ids = torch.zeros((num_patients, max_num_events, max_embedding_length), dtype=torch.long)
    rev_input_ids = torch.zeros((num_patients, max_num_events, max_embedding_length), dtype=torch.long)
    attention_masks = torch.zeros((num_patients, max_num_events, max_embedding_length), dtype=torch.long)
    rev_attention_masks = torch.zeros((num_patients, max_num_events, max_embedding_length), dtype=torch.long)
    token_type_ids = torch.zeros((num_patients, max_num_events, max_embedding_length), dtype=torch.long)
    rev_token_type_ids = torch.zeros((num_patients, max_num_events, max_embedding_length), dtype=torch.long)
    masks = torch.zeros((num_patients, max_num_events), dtype=torch.bool)
    rev_masks = torch.zeros((num_patients, max_num_events), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):
        # Patient 3 tensors (#events, sentence_length+pad)
        j_visits = min(patient['input_ids'].shape[0], max_num_events)
        w_len = min(patient['input_ids'].shape[1], max_embedding_length)
        iids = patient['input_ids']
        ams = patient['attention_masks']
        ttids = patient['token_type_ids']
        assert(iids.shape == ams.shape)
        assert(iids.shape == ttids.shape)
        """
        TODO: update `x`, `rev_x`, `masks`, and `rev_masks`
        """
        input_ids[i_patient, :j_visits, :w_len] = iids[:j_visits, :w_len].unsqueeze(0)
        attention_masks[i_patient, :j_visits, :w_len] = ams[:j_visits, :w_len].unsqueeze(0)
        token_type_ids[i_patient, :j_visits, :w_len] = ttids[:j_visits, :w_len].unsqueeze(0)
        # The tensor is (seq_length, sentence_length+pad). Leave embeddings,
        # flip temporal order of code/event sequence.
        rev_input_ids[i_patient, :j_visits, :iids.shape[-1]] = torch.flip(iids, dims=[0]).unsqueeze(0)[:,:j_visits, :w_len]
        rev_attention_masks[i_patient, :j_visits, :ams.shape[-1]] = torch.flip(ams, dims=[0]).unsqueeze(0)[:,:j_visits, :w_len]
        rev_token_type_ids[i_patient, :j_visits, :ttids.shape[-1]] = torch.flip(ttids, dims=[0]).unsqueeze(0)[:,:j_visits, :w_len]
        masks[i_patient, :j_visits] = 1
        rev_masks[i_patient, :j_visits] = 1
      
        # TODO(botelho3) - comment this out to reduce spew.
        # if i_patient == 0:
        #     print(f"------ p: {i_patient} ------")
        #     print(x[i_patient, :, :25])
        #     print(rev_x[i_patient, :, :25])
        #     print(masks[i_patient, :, :25])
        #     print(rev_masks[i_patient, :, :25])
        
    # print(f'Bert Fine Tune Collate\n'
    #       f'seq {sequences[0]["input_ids"].shape}, '
    #       f'seq {sequences[-1]["input_ids"].shape}\n'
    #       f'input_ids {input_ids.shape}, '
    #       f'rev_attention_masks {rev_attention_masks.shape}')
    x = {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'token_type_ids': token_type_ids,
    }
    rev_x = {
        'input_ids': rev_input_ids,
        'attention_mask': rev_attention_masks,
        'token_type_ids': rev_token_type_ids,
    }
    return {
        'x': x,
        'rev_x': rev_x,
        'masks': masks,
        'rev_masks': rev_masks,
        'y': y,
    }

In [ ]:
# Create the transform that will take each sample (visit) in the dataset
# and convert the text description of the visit into a single embedding.
bert_ft_xform = BertFineTuneTransform()
BERT_FT_EMBEDDING_SIZE = bert_ft_xform.emb_size

# Mortality Task Datasets: set_task -> SampleEHRDataset(SampleBaseDataset)
# Add a transform to convert the visit into an embedding.
MORT_DEMBFT_CODE2IDX = {}
MORT_DEMBFT_CODE_COUNT = {}
task_fn = functools.partial(mortality_pred_task_demb, MORT_DEMBFT_CODE_COUNT)
mortality_dembft_ds = mimic3base.set_task(task_fn, task_name=mortality_pred_task_demb.__name__)
MORT_DEMBFT_CODE2IDX = {
    code: idx for idx, code in enumerate(sorted(MORT_DEMBFT_CODE_COUNT.keys()))
}


indices = list(range(0, min(10000, len(mortality_dembft_ds)) ))
print(len(mortality_dembft_ds))
print(len(torch.utils.data.Subset(mortality_dembft_ds, indices)))

print(f"MORT_DEMB_CODE2IDX {len(MORT_DEMBFT_CODE2IDX)}")
# Wrap the default pyHealth dataset class in our own wrapper. The wrapper takes each
# sample and applies BERT to xform text->pytorch.tensor.
tmp = TextEmbedDataset(mortality_dembft_ds, transform=bert_ft_xform, should_cache=False)
mortality_dembft_dataset = torch.utils.data.Subset(tmp, indices)
print(len(mortality_dembft_dataset))

In [ ]:
if True:
    # Verify DataLoader properties.
    # Quick test without running the whole RNN training process.
    mort_dembft_loader = DataLoader(
        mortality_dembft_dataset,
        batch_size=BATCH_SIZE_,
        shuffle=SHUFFLE_,
        collate_fn=bert_fine_tune_collate,
    )
    # loader = DataLoader(mort_, batch_size=10, collate_fn=collate_fn)
    loader_iter = iter(mort_dembft_loader)
    # for _ in loader_iter:
    #     pass
    try:
        x, masks, rev_x, rev_masks, y = next(loader_iter)
    except StopIteration as e:
        print(e)

In [ ]:
# Free some memory
del bert_ft_xform
del mortality_dembft_ds
del mortality_dembft_dataset
gc.collect()
del bert_xform
del mortality_demb_dataset

# Model Training Loop

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, roc_curve, precision_recall_curve


def eval_model(model, val_loader):
    
    """
    TODO: evaluate the model.
    
    Arguments:
        model: the RNN model
        val_loader: validation dataloader
        
    Outputs:
        precision: overall precision score
        recall: overall recall score
        f1: overall f1 score
        roc_auc: overall roc_auc score
        
    Note that please pass all four arguments to the model so that we can use this function for both 
    models. (Use `model(x, masks, rev_x, rev_masks)`.)
        
    HINT: checkout https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
    """
    
    model.eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    model.eval()
    for x, masks, rev_x, rev_masks, y in val_loader:
        y_hat = model(x, masks, rev_x, rev_masks)
        y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
        y_hat = (y_hat > 0.5).int()
        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
    """
    TODO:
        Calculate precision, recall, f1, and roc auc scores.
        Use `average='binary'` for calculating precision, recall, and fscore.
    """
    # print(f'y_pred: {torch.sum(y_pred)}, y_true {torch.sum(y_true)}\n'
    #       f'y_score\n{y_score}\n'
    #       f'y_pred\n{y_pred}\n'
    #       f'y_true\n{y_true}\n')
    
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    roc_auc = roc_auc_score(y_true, y_score)
    rcurve = roc_curve(y_true, y_score)
    precision_curve, recall_curve, thresholds = precision_recall_curve(y_true, y_score)
    
    

    # your code here
    return p, r, f, roc_auc, rcurve, precision_curve, recall_curve

In [ ]:
def train(model, train_loader, val_loader, n_epochs, criterion, optimizer):
    """
    TODO: train the model.
    
    Arguments:
        model: the RNN model
        train_loader: training dataloder
        val_loader: validation dataloader
        n_epochs: total number of epochs
        
    You need to call `eval_model()` at the end of each training epoch to see how well the model performs 
    on validation data.
        
    Note that please pass all four arguments to the model so that we can use this function for both 
    models. (Use `model(x, masks, rev_x, rev_masks)`.)
    """
    
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, masks, rev_x, rev_masks, y in train_loader:
            """
            TODO:
                1. zero grad
                2. model forward
                3. calculate loss
                4. loss backward
                5. optimizer step
            """
            optimizer.zero_grad()
            outputs = model(x, masks, rev_x, rev_masks)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            # your code here
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        p, r, f, roc_auc, rcurve, _, _ = eval_model(model, val_loader)
        print('Epoch: {} \t Validation p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'
              .format(epoch+1, p, r, f, roc_auc))

### Model: CodeEmb

In [ ]:
# load the model here
mort_code_naive_rnn = CembRNN(num_codes = len(MORT_CEMB_CODE2IDX))
mort_code_naive_rnn

### Model: DescEmb

In [ ]:
# load the model here
mort_desc_naive_rnn = DembRNN()
mort_desc_naive_rnn

### Train: CodeEmb

In [ ]:
print('Training CodeEmb.')

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(mort_code_naive_rnn.parameters(), lr=0.001)
# number of epochs to train the model
n_epochs =10 
train(mort_code_naive_rnn, mort_cemb_train_loader, mort_cemb_val_loader,
      n_epochs, criterion, optimizer)

In [ ]:
p, r, f, roc_auc, rcurve, prec_curve, rec_curve = eval_model(mort_code_naive_rnn, mort_cemb_val_loader)
print('Validation Set p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'
      .format(p, r, f, roc_auc))

plt.plot(rcurve[0], rcurve[1])
plt.show()

plt.plot(rec_curve, prec_curve)
plt.show()

In [ ]:
p, r, f, roc_auc, rcurve,  prec_curve, rec_curve = eval_model(mort_code_naive_rnn, mort_cemb_test_loader)
print('Test Set p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'
      .format(p, r, f, roc_auc))

plt.plot(rcurve[0], rcurve[1])
plt.show()

plt.plot(rec_curve, prec_curve)
plt.show()

### Train: DescEmb

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(mort_desc_naive_rnn.parameters(), lr=0.001)

In [ ]:
# number of epochs to train the model
n_epochs = 5
train(mort_desc_naive_rnn, mort_demb_train_loader, mort_demb_val_loader,
      n_epochs, criterion, optimizer)

In [ ]:
p, r, f, roc_auc, rcurve,  prec_curve, rec_curve = eval_model(mort_desc_naive_rnn, mort_demb_val_loader)
print('Validation Set p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'
      .format(p, r, f, roc_auc))

plt.plot(rcurve[0], rcurve[1])
plt.show()

plt.plot(rec_curve, prec_curve)
plt.show()

In [ ]:
p, r, f, roc_auc, rcurve,  prec_curve, rec_curve = eval_model(mort_desc_naive_rnn, mort_demb_test_loader)
print('Test Set p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'
      .format(p, r, f, roc_auc))

plt.plot(rcurve[0], rcurve[1])
plt.show()

plt.plot(rec_curve, prec_curve)
plt.show()

# Condensed Training using Trainer

### Plot Utils

In [ ]:
embed_model_types = ['code_emb', 'desc_emb', 'desc_emb_ft']
predict_model_types = ['code_emb', 'desc_emb', 'desc_emb_ft']
task_types = ['mort', 'readm']

def PlotAucRecallResults(val_results, test_results, emb_type: str, task_type: str):
    ''' Plot AUC-ROC curve and P-R curve.
    
        val_results: validation set
        test_results: test set
    '''
    p, r, f, roc_auc, rcurve, precision_curve, recall_curve, acc = val_results
    plt.plot(rcurve[0], rcurve[1])
    plt.title(' '.join(['ROC Curve / AUC', emb_type, 'for task:', task_type]))
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.show()

    plt.plot(recall_curve, precision_curve)
    plt.title('PR Curve')
    plt.title(' '.join(['PR Curve', emb_type, task_type]))
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.show() 
#     p, r, f, roc_auc, rcurve, precision_curve, recall_curve, acc = test_results
#     plt.plot(rcurve[0], rcurve[1])
#     plt.title(' '.join(['ROC Curve / AUC', emb_type, 'for task:', task_type]))
#     plt.xlabel('FPR')
#     plt.ylabel('TPR')
#     plt.show()

#     plt.plot(recall_curve, precision_curve)
#     plt.title(' '.join(['PR Curve', emb_type, task_type]))
#     plt.xlabel('Recall')
#     plt.ylabel('Precision')
#     plt.show() 

    
def PlotDiffResults(results_a, results_b, task_type: str, labels: List[str]):
    results_set_labels = ['validation', 'test']
    for i in range(len(results_set_labels)):
        # ROC Curves for val, test.
        p, r, f, roc_auc, rcurve, precision_curve, recall_curve, acc = results_a[i][-1]
        plt.plot(rcurve[0], rcurve[1], label=labels[0] + '_' + results_set_labels[i])
        p, r, f, roc_auc, rcurve, precision_curve, recall_curve, acc = results_b[i][-1]
        plt.plot(rcurve[0], rcurve[1], label=labels[1] + '_' + results_set_labels[i])
        plt.title(' '.join(['ROC Curve / AUC', 'for task:', task_type]))
        plt.xlabel('FPR')
        plt.ylabel('TPR')
        plt.legend()
        plt.show()

        # PR Curves for val, test.
        p, r, f, roc_auc, rcurve, precision_curve, recall_curve, acc = results_a[i][-1]
        plt.plot(recall_curve, precision_curve, label=labels[0] + '_' + results_set_labels[i])
        p, r, f, roc_auc, rcurve, precision_curve, recall_curve, acc = results_b[i][-1]
        plt.plot(recall_curve, precision_curve, label=labels[1] + '_' + results_set_labels[i])
        plt.title(' '.join(['PR Curve', 'for task:', task_type]))
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.legend()
        plt.show() 

        
def PlotPrecisionRecallAcrossEpoch(val_results, test_results):
    num_epochs = len(val_results)
    epochs = range(num_epochs)
    results_set_labels = ['validation', 'test']
    
    recall_val = []
    precision_val = []
    recall_test = []
    precision_test = []
    for i in epochs:
        recall_val.append(val_results[i][1])
        precision_val.append(val_results[i][0])
        recall_test.append(test_results[i][1])
        precision_test.append(test_results[i][0])
   
    plt.plot(epochs, recall_val, label='_'.join(['recall', results_set_labels[0]]))
    plt.plot(epochs, recall_test, label='_'.join(['recall', results_set_labels[1]]))
    plt.plot(epochs, precision_val, label='_'.join(['precision', results_set_labels[0]]))
    plt.plot(epochs, precision_test, label='_'.join(['precision', results_set_labels[1]]))
    plt.title(' '.join(['Recall/Precision vs epoch']))
    plt.xlabel('Epoch')
    plt.ylabel('Recall/Precision')
    plt.legend()
    plt.show()
    

def PlotPrecisionRecallDiffAcrossEpoch(results_a, results_b):
    assert(len(results_a) == len(results_b))
    num_epochs = len(results_a)
    epochs = range(num_epochs)
    labels = ['a', 'b']
    results_set_labels = ['validation', 'test']
    
    recall_val_a = []
    precision_val_a = []
    recall_test_a = []
    precision_test_a = []
    recall_val_b = []
    precision_val_b = []
    recall_test_b = []
    precision_test_b = []
    for i in epochs:
        recall_val_a.append(results_a[i][1])
        precision_val_a.append(results_a[i][0])
        recall_test_a.append(results_a[i][1])
        precision_test_a.append(results_a[i][0])
        recall_val_b.append(results_b[i][1])
        precision_val_b.append(results_b[i][0])
        recall_test_b.append(results_b[i][1])
        precision_test_b.append(results_b[i][0])
   
    plt.plot(epochs, recall_val_a, label='_'.join(['recall', labels[0], results_set_labels[0]]))
    plt.plot(epochs, recall_test_a, label='_'.join(['recall', labels[0], results_set_labels[1]]))
    plt.plot(epochs, recall_val_b, label='_'.join(['recall', labels[1], results_set_labels[0]]))
    plt.plot(epochs, recall_test_b, label='_'.join(['recall', labels[1], results_set_labels[1]]))
    plt.title(' '.join(['Recall vs Epoch']))
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.legend()
    plt.show()
    
    
    plt.plot(epochs, precision_val_a, label='_'.join(['precision', labels[0], results_set_labels[0]]))
    plt.plot(epochs, precision_test_a, label='_'.join(['precision', labels[0], results_set_labels[1]]))
    plt.plot(epochs, precision_val_b, label='_'.join(['precision', labels[1], results_set_labels[0]]))
    plt.plot(epochs, precision_test_b, label='_'.join(['precision', labels[1], results_set_labels[1]]))
    plt.title(' '.join(['Precision vs Epoch']))
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.legend()
    plt.show()


def PlotAccuracyAcrossEpoch(results_a, results_b=None):
    if results_b:
        assert(len(results_a[0]) == len(results_b[0]))
    num_epochs = len(results_a[0])
    epochs = list(range(num_epochs))
    task_labels = ['a', 'b']
    results_set_labels = ['validation', 'test']
    
    accuracy_a_val = []
    accuracy_a_test = []
    accuracy_b_val = []
    accuracy_b_test = []
    for i in epochs:
        if results_a is not None:
            accuracy_a_val.append(results_a[0][i][7])
            accuracy_a_test.append(results_a[1][i][7])
        if results_b is not None:
            accuracy_b_val.append(results_b[0][i][7])
            accuracy_b_test.append(results_b[1][i][7])
            
    plt.plot(epochs, accuracy_a_val, label='_'.join(['recall', task_labels[0], results_set_labels[0]]))
    plt.plot(epochs, accuracy_a_test, label='_'.join(['recall', task_labels[0], results_set_labels[1]]))
    if results_b is not None:
        plt.plot(epochs, accuracy_b_val, label='_'.join(['accuracy', task_labels[1], results_set_labels[0]]))
        plt.plot(epochs, accuracy_b_test, label='_'.join(['accuracy', task_labels[1], results_set_labels[1]]))
    plt.title(' '.join(['Accuracy vs Epoch']))
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    
def PrintTrainTime(times):
    print('Total train time: {:.2} s'.format(sum(times)))
    print('Per epoch times: ' + ', '.join(['{:.2}'.format(t) for t in times]))


### CodeEmb - Mort - Dev

In [ ]:
import importlib
import trainlib
import models
import datasets
importlib.reload(trainlib)
importlib.reload(models)
importlib.reload(datasets)
from trainlib import Trainer
from models import ModelA, EHRModel, CembRNN, DembRNN
from datasets import SimpleDataset, DatasetCacher


embed_model_types = ['code_emb', 'desc_emb', 'desc_emb_ft']
predict_model_types = ['code_emb', 'desc_emb', 'desc_emb_ft']
task_types = ['mort', 'readm']
# Dev set.
# MORT_DEMB_CODE2IDX 1625
# MORT_CEMB_CODE2IDX 1456
# READM_DEMB_CODE2IDX 1523
# READM_CEMB_CODE2IDX 1456
# Full set.
# READMISSION_PER_PATIENT_ICD_9_CODE2IDX_ len: 6555
# MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_ len: 6732
# Full set.
# MORT_DEMB_CODE2IDX 6732
# MORT_CEMB_CODE2IDX 6546
# READM_CEMB_CODE2IDX 6546
# READM_DEMB_CODE2IDX 6555

args = DotArgs()
args.save_dir = os.path.join(os.getcwd(), 'modelcache')
if not os.path.exists(args.save_dir):
    os.mkdir(args.save_dir)
args.save_prefix = 'checkpoints'
args.random_seed = 90210
args.task = 'mort'
args.is_dev = True  # if dev mode on we use a small subset of the full dataset.
assert(args.task in task_types)

# Model Args
# Load either a code_emb embed model or a desc_emb embed model.
args.embed_model_type = 'code_emb'
args.predict_model_type = 'code_emb'
args.collate_fn=None
args.embed_index_size = 1456 # mort_cemb_metadata['extra_data']['embed_index_size']
args.pred_embed_size = 128

# Training Args
args.load_pretrained_weights = False
args.n_epochs = 10
args.learning_rate = 1.0e-3

In [ ]:
trainer = Trainer(args)
cemb_mort_dev_results = trainer.train()

In [ ]:
# One entry per epoch in cemb_mort_results.
val_set_results = cemb_mort_dev_results[0]
test_set_results = cemb_mort_dev_results[1]
PrintTrainTime(cemb_mort_dev_results[2])
PlotAucRecallResults(val_set_results[-1], test_set_results[-1],
                     emb_type='Cemb', task_type='Mortality')
PlotPrecisionRecallAcrossEpoch(val_set_results, test_set_results)
PlotAccuracyAcrossEpoch(cemb_mort_dev_results)

### DescEmb - Mort - Dev

In [ ]:
import importlib
import trainlib
import models
import datasets
importlib.reload(trainlib)
importlib.reload(models)
importlib.reload(datasets)
from trainlib import Trainer
from models import ModelA, EHRModel, CembRNN, DembRNN
from datasets import SimpleDataset, DatasetCacher

# MORT_DEMB_CODE2IDX 1625
# MORT_CEMB_CODE2IDX 1456

args = DotArgs()
args.save_dir = os.path.join(os.getcwd(), 'modelcache')
if not os.path.exists(args.save_dir):
    os.mkdir(args.save_dir)
args.save_prefix = 'checkpoints'
args.random_seed = 90210
args.task = 'mort'
args.is_dev = True  # if dev mode on we use a small subset of the full dataset.
assert(args.task in task_types)

# Model Args
# Load either a code_emb embed model or a desc_emb embed model.
args.embed_model_type = 'desc_emb'
args.predict_model_type = 'desc_emb'
args.override_batch_size = BATCH_SIZE_
args.collate_fn = bert_per_patient_collate_function_new_trainer
args.embed_index_size = 0 # unused mort_demb_metadata['extra_data']['embed_index_size']=768
args.pred_embed_size = 0  # unused 

# Training Args
args.load_pretrained_weights = False
args.n_epochs = 10
args.learning_rate = 1.0e-3

In [ ]:
trainer = Trainer(args)
demb_mort_dev_results = trainer.train()
del trainer
gc.collect()

In [ ]:
# One entry per epoch in demb_mort_results.
val_set_results = demb_mort_dev_results[0]
test_set_results = demb_mort_dev_results[1]
PrintTrainTime(demb_mort_dev_results[2])
PlotAucRecallResults(val_set_results[-1], test_set_results[-1],
                     emb_type='Demb', task_type='Mortality')
PlotPrecisionRecallAcrossEpoch(val_set_results, test_set_results)
PlotAccuracyAcrossEpoch(demb_mort_dev_results)
labels = ['cemb', 'demb']
PlotDiffResults(cemb_mort_dev_results, demb_mort_dev_results, task_type='Mortality', labels=labels)


### CodeEmb - Readm - Dev

In [ ]:
import importlib
import trainlib
import models
import datasets
importlib.reload(trainlib)
importlib.reload(models)
importlib.reload(datasets)
from trainlib import Trainer
from models import ModelA, EHRModel, CembRNN, DembRNN
from datasets import SimpleDataset, DatasetCacher

# Dev set.
# MORT_DEMB_CODE2IDX 1625
# MORT_CEMB_CODE2IDX 1456
# READM_DEMB_CODE2IDX 1523
# READM_CEMB_CODE2IDX 1456
# Full set.
# READMISSION_PER_PATIENT_ICD_9_CODE2IDX_ len: 6555
# MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_ len: 6732
# Full set.
# MORT_DEMB_CODE2IDX 6732
# MORT_CEMB_CODE2IDX 6546

args = DotArgs()
args.save_dir = os.path.join(os.getcwd(), 'modelcache')
if not os.path.exists(args.save_dir):
    os.mkdir(args.save_dir)
args.save_prefix = 'checkpoints'
args.random_seed = 90210
args.task = 'readm'
args.is_dev = True  # if dev mode on we use a small subset of the full dataset.
assert(args.task in task_types)

# Model Args
# Load either a code_emb embed model or a desc_emb embed model.
args.embed_model_type = 'code_emb'
args.predict_model_type = 'code_emb'
args.collate_fn=None
args.embed_index_size = 1456
args.pred_embed_size = 128

# Training Args
args.load_pretrained_weights = False
args.n_epochs = 10
args.learning_rate = 1.0e-3

In [ ]:
trainer = Trainer(args)
cemb_readm_dev_results = trainer.train()

In [ ]:
# One entry per epoch in cemb_mort_results.
val_set_results = cemb_readm_dev_results[0]
test_set_results = cemb_readm_dev_results[1]
PrintTrainTime(cemb_readm_dev_results[2])
PlotAucRecallResults(val_set_results[-1], test_set_results[-1],
                     emb_type='Cemb', task_type='Readmission')
PlotPrecisionRecallAcrossEpoch(val_set_results, test_set_results)
PlotAccuracyAcrossEpoch(cemb_readm_dev_results)

### DescEmb - Readm - Dev

In [ ]:
import importlib
import trainlib
import models
import datasets
importlib.reload(trainlib)
importlib.reload(models)
importlib.reload(datasets)
from trainlib import Trainer
from models import ModelA, EHRModel, CembRNN, DembRNN
from datasets import SimpleDataset, DatasetCacher

args = DotArgs()
args.save_dir = os.path.join(os.getcwd(), 'modelcache')
if not os.path.exists(args.save_dir):
    os.mkdir(args.save_dir)
args.save_prefix = 'checkpoints'
args.random_seed = 90210
args.task = 'readm'
args.is_dev = True  # if dev mode on we use a small subset of the full dataset.
assert(args.task in task_types)

# Model Args
# Load either a code_emb embed model or a desc_emb embed model.
args.embed_model_type = 'desc_emb'
args.predict_model_type = 'desc_emb'
args.override_batch_size = BATCH_SIZE_
args.collate_fn = bert_per_patient_collate_function_new_trainer
args.embed_index_size = 0 # unused mort_demb_metadata['extra_data']['embed_index_size']=768
args.pred_embed_size = 0  # unused 

# Training Args
args.load_pretrained_weights = False
args.n_epochs = 10
args.learning_rate = 1.0e-3

In [ ]:
trainer = Trainer(args)
demb_readm_dev_results = trainer.train()
del trainer
gc.collect()

In [ ]:
# One entry per epoch in demb_mort_results.
val_set_results = demb_readm_dev_results[0]
test_set_results = demb_readm_dev_results[1]
PrintTrainTime(demb_readm_dev_results[2])
PlotAucRecallResults(val_set_results[-1], test_set_results[-1],
                     emb_type='Demb', task_type='Mortality')
PlotPrecisionRecallAcrossEpoch(val_set_results, test_set_results)
PlotAccuracyAcrossEpoch(demb_readm_dev_results)
labels = ['cemb', 'demb']
PlotDiffResults(cemb_readm_dev_results, demb_readm_dev_results, task_type='Mortality', labels=labels)

### CodeEmb - Mort - Full

In [ ]:
import importlib
import trainlib
import models
import datasets
importlib.reload(trainlib)
importlib.reload(models)
importlib.reload(datasets)
from trainlib import Trainer
from models import ModelA, EHRModel, CembRNN, DembRNN
from datasets import SimpleDataset, DatasetCacher

# Dev set.
# MORT_DEMB_CODE2IDX 1625
# MORT_CEMB_CODE2IDX 1456
# Full set.
# READMISSION_PER_PATIENT_ICD_9_CODE2IDX_ len: 6555
# MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_ len: 6732
# Full set.
# MORT_DEMB_CODE2IDX 6732
# MORT_CEMB_CODE2IDX 6546

args = DotArgs()
args.save_dir = os.path.join(os.getcwd(), 'modelcache')
if not os.path.exists(args.save_dir):
    os.mkdir(args.save_dir)
args.save_prefix = 'checkpoints'
args.random_seed = 90210
args.task = 'mort'
args.is_dev = False  # if dev mode on we use a small subset of the full dataset.
assert(args.task in task_types)

# Model Args
# Load either a code_emb embed model or a desc_emb embed model.
args.embed_model_type = 'code_emb'
args.predict_model_type = 'code_emb'
args.collate_fn=None
args.embed_index_size = 6546  # mort_cemb_metadata['extra_data']['embed_index_size']
args.pred_embed_size = 128

# Training Args
args.load_pretrained_weights = False
args.n_epochs = 10
args.learning_rate = 1.0e-4

In [ ]:
trainer = Trainer(args)
cemb_mort_full_results = trainer.train()
del trainer
gc.collect()

In [ ]:
# One entry per epoch in cemb_mort_results.
val_set_results = cemb_mort_full_results[0]
test_set_results = cemb_mort_full_results[1]
PrintTrainTime(cemb_mort_full_results[2])
PlotAucRecallResults(val_set_results[-1], test_set_results[-1],
                     emb_type='Cemb', task_type='Mortality')
PlotPrecisionRecallAcrossEpoch(val_set_results, test_set_results)
PlotAccuracyAcrossEpoch(cemb_mort_full_results)

### DescEmb - Mort - Full

In [ ]:
import importlib
import trainlib
import models
import datasets
importlib.reload(trainlib)
importlib.reload(models)
importlib.reload(datasets)
from trainlib import Trainer
from models import ModelA, EHRModel, CembRNN, DembRNN
from datasets import SimpleDataset, DatasetCacher

# MORT_DEMB_CODE2IDX 1625
# MORT_CEMB_CODE2IDX 1456

args = DotArgs()
args.save_dir = os.path.join(os.getcwd(), 'modelcache')
if not os.path.exists(args.save_dir):
    os.mkdir(args.save_dir)
args.save_prefix = 'checkpoints'
args.random_seed = 90210
args.task = 'mort'
args.is_dev = False  # if dev mode on we use a small subset of the full dataset.
assert(args.task in task_types)

# Model Args
# Load either a code_emb embed model or a desc_emb embed model.
args.embed_model_type = 'desc_emb'
args.predict_model_type = 'desc_emb'
args.override_batch_size = BATCH_SIZE_
args.collate_fn = bert_per_patient_collate_function_new_trainer 
args.embed_index_size = 0 # unused mort_demb_metadata['extra_data']['embed_index_size']=768
args.pred_embed_size = 0  # unused 

# Training Args
args.load_pretrained_weights = False
args.n_epochs = 10
args.learning_rate = 1.0e-4

In [ ]:
trainer = Trainer(args)
demb_mort_full_results = trainer.train()
del trainer
gc.collect()

In [ ]:
# One entry per epoch in cemb_mort_results.
val_set_results = demb_mort_full_results[0]
test_set_results = demb_mort_full_results[1]
PrintTrainTime(demb_mort_full_results[2])
PlotAucRecallResults(val_set_results[-1], test_set_results[-1],
                     emb_type='Demb', task_type='Mortality')
PlotPrecisionRecallAcrossEpoch(val_set_results, test_set_results)
PlotAccuracyAcrossEpoch(demb_mort_full_results)
labels = ['cemb', 'demb']
PlotDiffResults(cemb_mort_full_results, demb_mort_full_results, task_type='Mortality', labels=labels)

### CodeEmb - Readm - Full 

In [ ]:
import importlib
import trainlib
import models
import datasets
importlib.reload(trainlib)
importlib.reload(models)
importlib.reload(datasets)
from trainlib import Trainer
from models import ModelA, EHRModel, CembRNN, DembRNN
from datasets import SimpleDataset, DatasetCacher

# Dev set.
# MORT_DEMB_CODE2IDX 1625
# MORT_CEMB_CODE2IDX 1456
# READM_DEMB_CODE2IDX 1523
# READM_CEMB_CODE2IDX 1456
# Full set.
# READMISSION_PER_PATIENT_ICD_9_CODE2IDX_ len: 6555
# MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_ len: 6732
# Full set.
# MORT_DEMB_CODE2IDX 6732
# MORT_CEMB_CODE2IDX 6546

args = DotArgs()
args.save_dir = os.path.join(os.getcwd(), 'modelcache')
if not os.path.exists(args.save_dir):
    os.mkdir(args.save_dir)
args.save_prefix = 'checkpoints'
args.random_seed = 90210
args.task = 'readm'
args.is_dev = False  # if dev mode on we use a small subset of the full dataset.
assert(args.task in task_types)

# Model Args
# Load either a code_emb embed model or a desc_emb embed model.
args.embed_model_type = 'code_emb'
args.predict_model_type = 'code_emb'
args.collate_fn=None
args.embed_index_size = 6546
args.pred_embed_size = 128

# Training Args
args.load_pretrained_weights = False
args.n_epochs = 10
args.learning_rate = 1.0e-4

In [ ]:
# 33426 / 32 * 0.8 = 837
trainer = Trainer(args)
cemb_readm_full_results = trainer.train()
del trainer
gc.collect()

In [ ]:
# One entry per epoch in cemb_mort_results.
val_set_results = cemb_readm_full_results[0]
test_set_results = cemb_readm_full_results[1]
PrintTrainTime(cemb_readm_full_results[2])
PlotAucRecallResults(val_set_results[-1], test_set_results[-1],
                     emb_type='Cemb', task_type='Readmission')
PlotPrecisionRecallAcrossEpoch(val_set_results, test_set_results)
PlotAccuracyAcrossEpoch(cemb_readm_full_results)

### DescEmb - Readm - Full 

In [ ]:
import importlib
import trainlib
import models
import datasets
importlib.reload(trainlib)
importlib.reload(models)
importlib.reload(datasets)
from trainlib import Trainer
from models import ModelA, EHRModel, CembRNN, DembRNN
from datasets import SimpleDataset, DatasetCacher

args = DotArgs()
args.save_dir = os.path.join(os.getcwd(), 'modelcache')
if not os.path.exists(args.save_dir):
    os.mkdir(args.save_dir)
args.save_prefix = 'checkpoints'
args.random_seed = 90210
args.task = 'readm'
args.is_dev = False  # if dev mode on we use a small subset of the full dataset.
assert(args.task in task_types)

# Model Args
# Load either a code_emb embed model or a desc_emb embed model.
args.embed_model_type = 'desc_emb'
args.predict_model_type = 'desc_emb'
args.override_batch_size = BATCH_SIZE_
args.collate_fn = bert_per_patient_collate_function_new_trainer
args.embed_index_size = 0 # unused mort_demb_metadata['extra_data']['embed_index_size']=768
args.pred_embed_size = 0  # unused 

# Training Args
args.load_pretrained_weights = False
args.n_epochs = 10
args.learning_rate = 1.0e-4

In [ ]:
# 10000 / 32 * 0.8 = 250
trainer = Trainer(args)
demb_readm_full_results = trainer.train()
del trainer
gc.collect()

In [ ]:
# One entry per epoch in demb_mort_results.
val_set_results = demb_readm_full_results[0]
test_set_results = demb_readm_full_results[1]
PrintTrainTime(demb_readm_full_results[2])
PlotAucRecallResults(val_set_results[-1], test_set_results[-1],
                     emb_type='Demb', task_type='Readmission')
PlotPrecisionRecallAcrossEpoch(val_set_results, test_set_results)
PlotAccuracyAcrossEpoch(demb_readm_full_results)
labels = ['cemb', 'demb']
PlotDiffResults(cemb_readm_full_results, demb_readm_full_results, task_type='Readmission', labels=labels)

### DescEmbFt - Mort - Dev

In [ ]:
import importlib
import trainlib
import models
import datasets
importlib.reload(trainlib)
importlib.reload(models)
importlib.reload(datasets)
from trainlib import Trainer
from models import ModelA, EHRModel, CembRNN, DembRNN
from datasets import SimpleDataset, DatasetCacher
import torch
# from GPUtil import showUtilization as gpu_usage

# MORT_DEMB_CODE2IDX 1625
# MORT_CEMB_CODE2IDX 1456

args = DotArgs()
args.save_dir = os.path.join(os.getcwd(), 'modelcache')
if not os.path.exists(args.save_dir):
    os.mkdir(args.save_dir)
args.save_prefix = 'checkpoints'
args.random_seed = 90210
args.task = 'mort'
args.is_dev = True  # if dev mode on we use a small subset of the full dataset.
assert(args.task in task_types)

# Model Args
# Load either a code_emb embed model or a desc_emb embed model.
args.embed_model_type = 'desc_emb_ft'
args.predict_model_type = 'desc_emb_ft'
args.override_batch_size = BATCH_SIZE_
args.collate_fn = bert_fine_tune_collate
args.no_use_cached_dataset = mortality_dembft_dataset
args.embed_index_size = 0 # unused mort_demb_metadata['extra_data']['embed_index_size']=768
args.pred_embed_size = 0  # unused 

# Training Args
args.load_pretrained_weights = False
args.n_epochs = 10
args.learning_rate = 1.0e-4

In [ ]:
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))
gc.collect()
del trainer
gc.collect()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
trainer = Trainer(args)
dembft_mort_dev_results = trainer.train()

In [ ]:
# One entry per epoch in demb_mort_results.
val_set_results = dembft_mort_dev_results[0]
test_set_results = dembft_mort_dev_results[1]
PrintTrainTime(dembft_mort_dev_results[2])
PlotAucRecallResults(val_set_results[-1], test_set_results[-1],
                     emb_type='Demb', task_type='Mortality')
PlotPrecisionRecallAcrossEpoch(val_set_results, test_set_results)
PlotAccuracyAcrossEpoch(dembft_mort_dev_results)

### DescEmbFt - Mort - Full 

In [ ]:
import importlib
import trainlib
import models
import datasets
importlib.reload(trainlib)
importlib.reload(models)
importlib.reload(datasets)
from trainlib import Trainer
from models import ModelA, EHRModel, CembRNN, DembRNN
from datasets import SimpleDataset, DatasetCacher


# MORT_DEMB_CODE2IDX 1625
# MORT_CEMB_CODE2IDX 1456

args = DotArgs()
args.save_dir = os.path.join(os.getcwd(), 'modelcache')
if not os.path.exists(args.save_dir):
    os.mkdir(args.save_dir)
args.save_prefix = 'checkpoints'
args.random_seed = 90210
args.task = 'mort'
args.is_dev = False  # if dev mode on we use a small subset of the full dataset.
assert(args.task in task_types)

# Model Args
# Load either a code_emb embed model or a desc_emb embed model.
args.embed_model_type = 'desc_emb_ft'
args.predict_model_type = 'desc_emb_ft'
args.override_batch_size = BATCH_SIZE_
args.collate_fn = bert_fine_tune_collate
args.no_use_cached_dataset = mortality_dembft_dataset
args.embed_index_size = 0 # unused mort_demb_metadata['extra_data']['embed_index_size']=768
args.pred_embed_size = 0  # unused 


# Training Args
args.load_pretrained_weights = False
args.n_epochs = 10
args.learning_rate = 1.0e-4

In [ ]:
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))
gc.collect()
del trainer
gc.collect()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
trainer = Trainer(args)
dembft_mort_full_results = trainer.train()

In [ ]:
# One entry per epoch in cemb_mort_results.
val_set_results = dembft_mort_full_results[0]
test_set_results = dembft_mort_full_results[1]
PrintTrainTime(dembft_mort_full_results[2])
PlotAucRecallResults(val_set_results[-1], test_set_results[-1],
                     emb_type='Demb', task_type='Mortality')
PlotPrecisionRecallAcrossEpoch(val_set_results, test_set_results)
PlotAccuracyAcrossEpoch(dembft_mort_full_results)

# eICU Load

In [ ]:
print(f'Reading data from: `{EICU_DATA_DIR_}`')

eicubase = eICUDataset(
    # root="https://storage.googleapis.com/pyhealth/Synthetic_MIMIC-III/",
    root=EICU_DATA_DIR_,
    dataset_name='eicu_dataset',
    tables=["diagnosis", "treatment", "medication"],
    # map all NDC codes to ATC 3-rd level codes in these tables
    # See https://en.wikipedia.org/wiki/Anatomical_Therapeutic_Chemical_Classification_System.
    code_mapping={"NDC": ("ATC", {"target_kwargs": {"level": 3}})},
    # Reads a subset of the data. Disable for full training run.
    dev = False,
    # True = Slow, rebuilds the dataset instead of caching.
    refresh_cache=False,
)

eicubase.stat()
eicubase.info()

#### CodeEMB Pred Task

In [ ]:
def eicu_mortality_pred_task_cemb(CODE_COUNT, patient):
    """
    patient is a <pyhealth.data.Patient> object
    each sample is a list of vists for 1 patient.
    """
   
    # TODO(botelho3) - stupid hack around the limitations of SampleDataset validator.
    kMaxVisits = 5
    if len(patient) < 1 or len(patient) > kMaxVisits:
        return []
        
    samples = [{
        "visit_id": patient[0].visit_id,
        "patient_id": patient.patient_id,
        # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
        "num_visits": 0,
        "conditions": [[] for v in range(kMaxVisits)],
        "procedures": [[] for v in range(kMaxVisits)],
        "conditions_text": [[] for v in range(kMaxVisits)],
        "procedures_text": [[] for v in range(kMaxVisits)],
        "drugs": [[] for v in range(kMaxVisits)],
        "drugs_text": [[] for v in range(kMaxVisits)],
        # "labevents": labevents,
        # "labevents_text": labevents_text
        "label": 0,
    }]
    
    global_mortality_label = 0
    # loop over all visits but the last one
    for i in range(len(patient)):

        # visit and next_visit are both <pyhealth.data.Visit> objects
        # there are no vists.attr_dict keys
        visit: Visit = patient[i]
        # next_visit: Visit = patient[i + 1]
        # if i == 0: print(visit)

        # step 1: define the mortality_label
        # if next_visit.discharge_status not in [0, 1]:
        #     mortality_label = 0
        # else:
        #     mortality_label = int(next_visit.discharge_status)
        mortality_label = int(visit.discharge_status)
        global_mortality_label |= mortality_label

    # loop over all visits
    out_idx = 0
    for i, visit in enumerate(patient):
        # visit and next_visit are both <pyhealth.data.Visit> objects
        # there are no vists.attr_dict keys

        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="diagnosis")
        procedures = visit.get_code_list(table="treatment")
        drugs = visit.get_code_list(table="medication")
        drugs_full = visit.get_event_list(table="medication")
        # if i == 0: print([d.attr_dict for d in drugs_full])
        # if i == 0: print(conditions)
        # if i == 0: print(procedures)
        # if i == 0: print(drugs)
        # TODO(botelho3) - add this datasource back in once we have full MIMIC-III dataset.
        # labevents = visit.get_code_list(table="LABEVENTS")

        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) * len(drugs_full) == 0:
            continue
        if len(conditions) + len(procedures) + len(drugs_full) < 5:
            # Exclude stays with less than 5 procedures.
            continue
        
        # step 3.5: build text lists from the ICD codes
        # diag_lut = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        # proc_lut = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        # if i == 0: print(diag_lut)
        # if i == 0: print(proc_lut)
        # Index 0 is shortname, index 1 is longname.
        # print([str(cond) + ' ' + str(diag_lut.get(cond)) for cond in conditions])
        # print(proc_lut.get(procedures[0]))
        print(f'condition {conditions}')
        print(f'proc {procedures}')
        print(f'drugs {drugs_full}')
        conditions_text = [diag_lut.get(cond,("", ""))[1] for cond in conditions]
        procedures_text = [proc_lut.get(proc,("", ""))[1] for proc in procedures]
        drugs_text = [' '.join([d['dname'], d['dtype'], str(d['dose']), d['route'], str(d['duration'])])
                      for d in [d.attr_dict for d in drugs_full]]
        # TODO(botelho3) - add the labevents data source back in once we have full MIMIC-III dataset.
        # labevents_text =
        
        # step 4: assemble the samples
        # Each field is a list of visits.
        samples[0]['num_visits'] = samples[0]['num_visits'] + 1
        samples[0]['conditions'][out_idx]=conditions
        samples[0]['procedures'][out_idx]=procedures
        samples[0]['conditions_text'][out_idx]= conditions_text
        samples[0]['procedures_text'][out_idx] = procedures_text
        samples[0]['drugs'][out_idx] = drugs
        samples[0]['drugs_text'][out_idx] = drugs_text
        out_idx = out_idx + 1
    samples[0]['label'] = global_mortality_label
   
    # Record all unique codes and their frequency for LUT.
    for visit in samples[0]['conditions']:
        for code in visit:
            CODE_COUNT[code] = CODE_COUNT.get(code, 0) + 1
           
    # If none of the samples met the criteria return an empty list.
    if samples[0]['num_visits'] == 0:
        return []
        
    return samples


#### DescEMB Pred Task

In [ ]:
re_drug_prefix = re.compile('Event with eICU_DRUGNAME code{.*}from table medication')
re_bar = re.compile('\|')
ICD_9_LUT_ = {}
ICD_10_LUT_ = {}
fname = os.path.expanduser('~/sw/icd10cm-code descriptions- April 1 2023/icd10cm-codes- April 1 2023.txt')
with open(fname, 'r') as f:
    for l in f:
        code, desc = l.split(sep=' ', maxsplit=1)
        ICD_10_LUT_[code] = desc
        
def eicu_mortality_pred_task_demb(CODE_COUNT, patient):
    """
    patient is a <pyhealth.data.Patient> object
    """
    samples = []
    visits = []
    kMaxListSize = 40
    
    global_mortality_label = 0
    # loop over all visits but the last one
    for i in range(len(patient)):

        # visit and next_visit are both <pyhealth.data.Visit> objects
        # there are no vists.attr_dict keys
        visit: Visit = patient[i]
        mortality_label = 0 if visit.discharge_status == 'Alive' else 1
        global_mortality_label |= mortality_label
        
    # loop over all visits but the last one
    for i, visit in enumerate(patient):
        # visit: Visit.
        
        # step 2: get code-based feature information
        conditions = visit.get_code_list(table="diagnosis")
        procedures = visit.get_code_list(table="treatment")
        # drugs = [x.code for x in visit.get_event_list(table="medication")]
        drugs_full = visit.get_event_list(table="medication")
        drugs_full = [d.code for d in drugs_full]
        # if i == 0: print([d.attr_dict for d in drugs_full])
        # if i == 0: print(conditions)
        # if i == 0: print(procedures)
        # if i == 0: print(drugs)
        # TODO(botelho3) - add this datasource back in once we have full MIMIC-III dataset.
        # labevents = visit.get_code_list(table="LABEVENTS")

        # step 3: exclusion criteria: visits without condition, procedure, or drug
        if len(conditions) * len(procedures) == 0 * len(drugs_full) == 0:
            # print(f'Excluded something 0 {len(conditions)}, {len(procedures)}, {len(drugs_full)}')
            # print(f'conditions {conditions}')
            # print(f'procedures {procedures}')
            # print(f'drugs_full {drugs_full}')
            continue
        if len(conditions) + len(procedures) + len(drugs_full) < 5:
            # Exclude stays with less than 5 procedures.
            continue
        
        # step 3.5: build text lists from the ICD codes
        # diag_lut = mimic3base.get_text_lut("D_ICD_DIAGNOSES")
        # proc_lut = mimic3base.get_text_lut("D_ICD_PROCEDURES")
        
        # if i == 0: print(d_diag)
        # if i == 0: print(d_proc)
        # Index 0 is shortname, index 1 is longname.
        # print([str(cond) + ' ' + str(d_diag.get(cond)) for cond in conditions])
        # print(d_proc.get(procedures[0]))
        # print(f'condition {conditions}')
        # print(f'proc {procedures}')
        # print(f'drugs {drugs_full}')
        # conditions_text = [diag_lut.get(cond,("", ""))[1] for cond in conditions]
        # procedures_text = [proc_lut.get(proc,("", ""))[1] for proc in procedures]
        conditions = filter(lambda x: True if x[0].isalpha() else False, conditions)
        conditions = [cond.replace('.', '') for cond in conditions]
        conditions_text = [ICD_10_LUT_.get(cond, '') for cond in conditions] 
        procedures_text = [re_bar.sub(' ', proc) for proc in procedures]
        drugs_text = [re_drug_prefix.sub('\1', str(d)) for d in drugs_full]
        # TODO(botelho3) - add the labevents data source back in once we have full MIMIC-III dataset.
        # labevents_text =
        
        # step 4: assemble the samples into a pyHealth Visit.
        visits.append(
            {
                "visit_id": visit.visit_id,
                "patient_id": patient.patient_id,
                # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
                "conditions": conditions,
                "procedures": procedures,
                "conditions_text": conditions_text,
                "procedures_text": procedures_text,
                "drugs_text": drugs_text,
                # "labevents": labevents,
                # "labevents_text": labevents_text
                "label": global_mortality_label,
            }
        )
   
    
    # Return empty list, didn't meet exclusion criteria.
    num_visits = len(visits)
    if num_visits < 1:
        return [] 
    
   
    # pyHealth requires that all list fields in sample are equal size.
    def pad_field(field, visits, empty_val: Any):
        l = [empty_val for x in range(kMaxListSize)]
        data = [x[field] for x in visits]
        data = list(itertools.chain.from_iterable(data))
        slice_size = min(kMaxListSize, len(data))
        l[:slice_size] = data[:slice_size]
        return l, slice_size
    
    conditions, conditions_pad = pad_field("conditions", visits, '0')
    conditions_text, conditions_text_pad = pad_field("conditions_text", visits, '')
    procedures, procedures_pad = pad_field("procedures", visits, '0')
    procedures_text, procedures_text_pad = pad_field("procedures_text", visits, '')
    drugs_text, drugs_text_pad = pad_field("drugs_text", visits, '')
    sample = {
        "patient_id": patient.patient_id,
        # TODO(botelho3) Why does pyhealth require a visit id in the keys if we're combining vists?
        "visit_id": visits[0]["visit_id"],
        "num_visits": num_visits,
        # the following keys can be the "feature_keys" or "label_key" for initializing downstream ML model
        "conditions": conditions,
        "conditions_text": conditions_text,
        "procedures": procedures,
        "procedures_text": procedures_text,
        "drugs_text": drugs_text,
        
        "conditions_pad": conditions_pad,
        "procedures_pad": procedures_pad,
        "conditions_text_pad": conditions_text_pad,
        "procedures_text_pad": procedures_text_pad,
        "drugs_text_pad": drugs_text_pad,
        # "labevents": labevents,
        # "labevents_text": labevents_text
        "label": global_mortality_label,
    }
   
    # For every condition in the sample (all visits). Record frequency.
    # Will be used to build code->index LUT.
    for code in sample['conditions']:
        CODE_COUNT[code] = CODE_COUNT.get(code, 0) + 1
        
    samples.append(sample)
    return samples

#### Test Load Mortality Dataset

In [ ]:
EICU_MORTALITY_PER_PATIENT_ICD_9_CODE_COUNT_ = {}
EICU_MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_ = {}
task_fn = functools.partial(eicu_mortality_pred_task_demb, EICU_MORTALITY_PER_PATIENT_ICD_9_CODE_COUNT_)
mor_dataset = eicubase.set_task(task_fn, task_name=eicu_mortality_pred_task_demb.__name__)
EICU_MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_ = {
    code: idx for idx, code in enumerate(sorted(EICU_MORTALITY_PER_PATIENT_ICD_9_CODE_COUNT_.keys()))
}
mor_dataset.stat()
mor_dataset.samples[1]
# TODO(botelho3) could try a freq codes limit on this.
# print(f"MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_ len: {len(EICU_MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_)}\n"
#       f"{EICU_MORTALITY_PER_PATIENT_ICD_9_CODE2IDX_}")
del mor_dataset

In [ ]:
def load_model_state_from_file(fname):
    path = os.path.join(os.getcwd(), 'modelcache', fname)
        #     torch.save(
        #     {
        #         # 'model_state_dict': self.model.module.state_dict() if (
        #         #     isinstance(self.model, DataParallel)
        #         # ) else self.model.state_dict(),
        #         'model_state_dict': self.model.state_dict(),
        #         'optimizer_state_dict': self.optimizer.state_dict(),
        #         'epochs': epoch,
        #         'args': self.args,
        #     },
        #     path
        # )
    d = torch.load(path)
    model_state_dict = d['model_state_dict']
    args = d['args']
    print(f'Loading model state with args{args} from \n {path}')
    return model_state_dict, args

def load_model_from_file(embed_model_type: str,
                         task: str,
                         is_dev: bool):
    fname = f'model_{embed_model_type}_task_{task}_isdev{is_dev}'
    fname = fname + "_best.pt"
    model_state_dict, args = load_model_state_from_file(fname)
    model = EHRModel(args)
    model.load_state_dict(model_state_dict)
    model.eval()
    return model, args
   
model = load_model_from_file('desc_emb_ft', 'mort', is_dev=False) 
print(model)

#### Load DescEmb - Mort - Full

#### Load DescEmbFt - Mort - Full